# Chapter 5: Classes and Interfaces

---

This chapter covers best practices for designing classes and interfaces in Python, including composition patterns, polymorphism, inheritance, and advanced attribute handling.

## Item 37: Compose Classes Instead of Nesting Built-in Types

### The Problem with Deep Nesting

Dictionaries, lists, tuples, and sets are easy to use, but there's a danger of **overextending them** to write brittle code. When your bookkeeping gets complicated, break it into classes.

### Example: Grade Tracking System Evolution

#### Version 1: Simple Dictionary

In [2]:
# Simple gradebook with overall grades
class SimpleGradebook:
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = []
    
    def report_grade(self, name, grade):
        self._grades[name].append(grade)
    
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

# Usage
book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton', 90)
book.report_grade('Isaac Newton', 95)
book.report_grade('Isaac Newton', 85)

print(f"Average: {book.average_grade('Isaac Newton')}")

Average: 90.0


#### Version 2: Adding Subjects (Nested Dictionary)

In [3]:
from collections import defaultdict

class BySubjectGradebook:
    def __init__(self):
        self._grades = {}  # Outer dict
    
    def add_student(self, name):
        self._grades[name] = defaultdict(list)  # Inner dict
    
    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append(grade)
    
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total / count

# Usage
book = BySubjectGradebook()
book.add_student('Albert Einstein')
book.report_grade('Albert Einstein', 'Math', 75)
book.report_grade('Albert Einstein', 'Math', 65)
book.report_grade('Albert Einstein', 'Gym', 90)
book.report_grade('Albert Einstein', 'Gym', 95)

print(f"Average: {book.average_grade('Albert Einstein')}")

Average: 81.25


#### Version 3: Adding Weights (Tuple in List) - TOO COMPLEX!

In [4]:
from collections import defaultdict

# ============================================================================
# COMPLEXITY ANALYSIS: WeightedGradebook Class
# ============================================================================
# This code demonstrates multiple layers of complexity that make it difficult
# to understand, maintain, and debug. Comments below identify each issue.
# ============================================================================


class WeightedGradebook:
    def __init__(self):
        # COMPLEXITY ISSUE #1: Three-level nested data structure
        # -------------------------------------------------------
        # Structure: dict -> defaultdict -> list -> tuple
        # 
        # _grades = {
        #     'Student Name': defaultdict(list) {
        #         'Subject': [(score, weight), (score, weight), ...]
        #     }
        # }
        #
        # Problem: Each level of nesting adds cognitive load. Developers must
        # mentally track three levels deep to understand where data lives.
        # This makes debugging painful - print statements need multiple levels
        # of unpacking to see actual values.
        self._grades = {}
    
    def add_student(self, name):
        # COMPLEXITY ISSUE #2: defaultdict hides initialization logic
        # -----------------------------------------------------------
        # Problem: Using defaultdict(list) makes the structure implicit.
        # Readers must understand defaultdict behavior to know that accessing
        # a non-existent subject automatically creates an empty list.
        # This "magic" initialization obscures the data structure.
        #
        # Also: We're nesting a defaultdict inside a regular dict, mixing
        # two different dictionary types, which adds mental overhead.
        self._grades[name] = defaultdict(list)
    
    def report_grade(self, name, subject, score, weight):
        # COMPLEXITY ISSUE #3: Multi-step data structure navigation
        # ----------------------------------------------------------
        # Problem: Requires two separate lookups to reach the final destination.
        # Each intermediate assignment creates a temporary variable that exists
        # only to make the next lookup possible.
        
        # Step 1: Navigate from student name to their subject dictionary
        by_subject = self._grades[name]
        
        # Step 2: Navigate from subject to the list of grade tuples
        grade_list = by_subject[subject]
        
        # COMPLEXITY ISSUE #4: Opaque tuple structure
        # -------------------------------------------
        # Problem: (score, weight) has no named fields. Readers must remember:
        # - Position 0 = score
        # - Position 1 = weight
        # 
        # If someone reverses these accidentally, there's no type safety or
        # field names to catch the error. The bug would be silent and hard
        # to detect.
        #
        # COMPLEXITY ISSUE #5: Data mutation at the deepest level
        # --------------------------------------------------------
        # Problem: We're appending to a list that's inside a defaultdict
        # that's inside a regular dict. This deep mutation makes it hard
        # to track where data changes happen in the codebase.
        grade_list.append((score, weight))
    
    def average_grade(self, name):
        # COMPLEXITY ISSUE #6: Initial navigation still required
        # -------------------------------------------------------
        # Problem: Every method must start by navigating the structure.
        # This repetitive pattern suggests the structure itself is too complex.
        by_subject = self._grades[name]
        
        # COMPLEXITY ISSUE #7: Multiple accumulator variables
        # ---------------------------------------------------
        # Problem: Tracking score_sum and score_count separately requires
        # readers to mentally compute the relationship between them.
        # It's not immediately obvious what these represent until you
        # reach the final return statement.
        score_sum, score_count = 0, 0
        
        # COMPLEXITY ISSUE #8: Nested loop structure
        # ------------------------------------------
        # Problem: Outer loop iterates subjects, inner loop iterates grades.
        # Each loop has its own accumulator variables, creating a matrix of
        # state that must be tracked simultaneously.
        for subject, scores in by_subject.items():
            # COMPLEXITY ISSUE #9: Inner accumulator variables
            # ------------------------------------------------
            # Problem: Now we have FOUR variables being tracked:
            # - score_sum (outer)
            # - score_count (outer)
            # - subject_avg (inner)
            # - total_weight (inner)
            #
            # Readers must understand which variables belong to which scope
            # and how they interact across loop boundaries.
            subject_avg, total_weight = 0, 0
            
            # COMPLEXITY ISSUE #10: Tuple unpacking in loop
            # ---------------------------------------------
            # Problem: (score, weight) unpacking looks simple, but it's
            # operating on the opaque tuple structure from Issue #4.
            # If the tuple structure changes, this breaks silently.
            for score, weight in scores:
                # COMPLEXITY ISSUE #11: Weighted average calculation buried in loops
                # ------------------------------------------------------------------
                # Problem: The actual mathematical formula is split across multiple
                # lines and mixed with loop control logic. It's hard to see that
                # this implements: weighted_avg = Σ(score × weight) / Σ(weight)
                subject_avg += score * weight
                total_weight += weight
            
            # COMPLEXITY ISSUE #12: Division without zero-check
            # -------------------------------------------------
            # Problem: If total_weight is 0, this crashes. The data structure
            # allows this invalid state, but there's no guard against it.
            # (Though in practice, add_student/report_grade make this unlikely)
            score_sum += subject_avg / total_weight
            score_count += 1
        
        # COMPLEXITY ISSUE #13: Final calculation obscured
        # ------------------------------------------------
        # Problem: By the time we reach this line, readers have mentally
        # executed two loops and tracked four variables. The simple fact that
        # we're computing "average of subject averages" is lost in complexity.
        return score_sum / score_count


# ============================================================================
# USAGE COMPLEXITY ISSUES
# ============================================================================

book = WeightedGradebook()
book.add_student('Albert Einstein')

# COMPLEXITY ISSUE #14: Unclear parameter meaning
# ------------------------------------------------
# Problem: What does 0.05 mean? What does 75 mean? Are these percentages?
# Decimals? The API provides no hints. You must read documentation or
# source code to understand that:
# - 75 is the score (but out of what? 100? 1000?)
# - 0.05 is the weight (but weights for what? Do they sum to 1.0?)
book.report_grade('Albert Einstein', 'Math', 75, 0.05)
book.report_grade('Albert Einstein', 'Math', 65, 0.15)
book.report_grade('Albert Einstein', 'Math', 70, 0.80)

# COMPLEXITY ISSUE #15: No validation of weight sums
# ---------------------------------------------------
# Problem: These weights (0.05 + 0.15 + 0.80 = 1.0) sum to 1.0 for Math,
# but (0.40 + 0.60 = 1.0) for Gym. This is correct, but nothing enforces it.
# If a user enters weights that sum to 0.7, the system silently accepts it
# and produces mathematically questionable results.
book.report_grade('Albert Einstein', 'Gym', 100, 0.40)
book.report_grade('Albert Einstein', 'Gym', 85, 0.60)

# COMPLEXITY ISSUE #16: Hidden calculation complexity
# ---------------------------------------------------
# Problem: This single line triggers:
# - 1 dictionary lookup
# - 1 iteration over subjects (defaultdict)
# - N iterations over grade tuples
# - Multiple arithmetic operations
# - 2 division operations
#
# None of this complexity is visible at the call site. The simple-looking
# method call hides significant computational work.
print(f"Average: {book.average_grade('Albert Einstein')}")


# ============================================================================
# SUMMARY OF COMPLEXITY ISSUES
# ============================================================================
# 
# STRUCTURAL COMPLEXITY:
# - Three-level nested data structure (dict -> defaultdict -> list -> tuple)
# - Mixed dictionary types (regular dict + defaultdict)
# - Opaque tuple structure without named fields
# - Deep mutation at the innermost level
#
# ALGORITHMIC COMPLEXITY:
# - Nested loops with multiple accumulator variables
# - Weighted average calculation split across multiple statements
# - No guard against division by zero (total_weight)
# - Mathematical formula obscured by control flow
#
# API COMPLEXITY:
# - Unclear parameter meanings (positional arguments with no hints)
# - No validation of weight constraints
# - Repetitive structure navigation in every method
# - Hidden computational complexity
#
# MAINTENANCE COMPLEXITY:
# - Difficult to debug (deep nesting requires multiple print statements)
# - Fragile to changes (tuple ordering must be preserved)
# - Hard to extend (adding features requires understanding all three levels)
# - No type hints to guide correct usage
#
# ============================================================================
# RECOMMENDATION: Refactor using named structures (namedtuple, dataclass)
# and flatten the data model to reduce nesting levels.
# ============================================================================

Average: 80.25


## IDEAL APPROACH: Refactored WeightedGradebook

In [5]:
"""
Refactored WeightedGradebook with Python 3.10+ Type Hints

This version uses modern Python 3.10+ features:
- Built-in generic types (list, dict) instead of typing.List, typing.Dict
- dataclasses with slots for memory efficiency
- match/case statements (Python 3.10+)
- Improved type annotations
"""

from dataclasses import dataclass


# ============================================================================
# IDEAL APPROACH: Refactored WeightedGradebook (Python 3.10+)
# ============================================================================


@dataclass(slots=True)  # slots=True for memory efficiency (Python 3.10+)
class Grade:
    """
    Represents a single grade with score and weight.
    
    Modern improvements:
    - Uses lowercase 'list' and 'dict' for type hints (Python 3.9+)
    - Uses slots for reduced memory footprint (Python 3.10+)
    - Validates data at creation time
    """
    score: float
    weight: float
    
    def __post_init__(self):
        """Validate grade data immediately upon creation."""
        if not 0 <= self.score <= 100:
            raise ValueError(f"Score must be 0-100, got {self.score}")
        if not 0 < self.weight <= 1:
            raise ValueError(f"Weight must be 0-1, got {self.weight}")


@dataclass(slots=True)
class Subject:
    """
    Represents a subject with multiple weighted grades.
    
    Type hints use lowercase 'list' (Python 3.9+ feature)
    """
    name: str
    grades: list[Grade]  # Modern: lowercase 'list' instead of typing.List
    
    def weighted_average(self) -> float:
        """
        Calculate weighted average for this subject.
        
        Formula: Σ(score × weight) / Σ(weight)
        """
        if not self.grades:
            return 0.0
        
        total_weighted_score = sum(g.score * g.weight for g in self.grades)
        total_weight = sum(g.weight for g in self.grades)
        
        if total_weight == 0:
            return 0.0
        
        return total_weighted_score / total_weight
    
    def validate_weights(self) -> bool:
        """
        Check if weights sum to approximately 1.0.
        
        Returns:
            True if weights sum to 1.0 (within floating-point tolerance)
        """
        total_weight = sum(g.weight for g in self.grades)
        return abs(total_weight - 1.0) < 0.0001
    
    def get_grade_count(self) -> int:
        """Return the number of grades in this subject."""
        return len(self.grades)


class Student:
    """
    Represents a student with multiple subjects.
    
    Modern type hints:
    - Uses lowercase 'dict' instead of typing.Dict (Python 3.9+)
    - Clear separation of concerns
    """
    
    def __init__(self, name: str):
        self.name = name
        self._subjects: dict[str, Subject] = {}  # Modern: lowercase 'dict'
    
    def add_grade(self, subject_name: str, score: float, weight: float) -> None:
        """
        Add a grade for a specific subject.
        
        Args:
            subject_name: Name of the subject
            score: Grade score (0-100)
            weight: Weight of this grade (0-1)
        
        Raises:
            ValueError: If score or weight are invalid
        """
        # Get or create subject
        if subject_name not in self._subjects:
            self._subjects[subject_name] = Subject(subject_name, [])
        
        # Create validated Grade object
        grade = Grade(score=score, weight=weight)
        
        # Add to subject
        self._subjects[subject_name].grades.append(grade)
    
    def average_grade(self) -> float:
        """
        Calculate overall average across all subjects.
        
        Returns:
            Average of all subject averages
        """
        if not self._subjects:
            return 0.0
        
        subject_averages = [
            subject.weighted_average() 
            for subject in self._subjects.values()
        ]
        
        return sum(subject_averages) / len(subject_averages)
    
    def get_subject(self, subject_name: str) -> Subject:
        """
        Retrieve a specific subject.
        
        Args:
            subject_name: Name of the subject to retrieve
        
        Returns:
            Subject object
        
        Raises:
            KeyError: If subject not found
        """
        if subject_name not in self._subjects:
            raise KeyError(f"Student {self.name} has no grades for {subject_name}")
        return self._subjects[subject_name]
    
    def has_subject(self, subject_name: str) -> bool:
        """Check if student has grades for a specific subject."""
        return subject_name in self._subjects
    
    def list_subjects(self) -> list[str]:
        """Return list of all subject names."""
        return list(self._subjects.keys())
    
    def validate_all_subjects(self) -> dict[str, bool]:
        """
        Validate weights for all subjects.
        
        Returns:
            Dictionary mapping subject names to validation status
        """
        return {
            name: subject.validate_weights() 
            for name, subject in self._subjects.items()
        }
    
    def get_subject_count(self) -> int:
        """Return the number of subjects this student has."""
        return len(self._subjects)


class Gradebook:
    """
    Container for managing multiple students.
    
    Modern type hints with lowercase 'dict' (Python 3.9+)
    """
    
    def __init__(self):
        self._students: dict[str, Student] = {}  # Modern: lowercase 'dict'
    
    def add_student(self, name: str) -> Student:
        """
        Add a new student to the gradebook.
        
        Args:
            name: Student name
        
        Returns:
            The created Student object
        
        Raises:
            ValueError: If student already exists
        """
        if name in self._students:
            raise ValueError(f"Student {name} already exists")
        
        student = Student(name)
        self._students[name] = student
        return student
    
    def get_student(self, name: str) -> Student:
        """
        Retrieve a student by name.
        
        Args:
            name: Student name
        
        Returns:
            Student object
        
        Raises:
            KeyError: If student not found
        """
        if name not in self._students:
            raise KeyError(f"Student {name} not found")
        return self._students[name]
    
    def has_student(self, name: str) -> bool:
        """Check if a student exists in the gradebook."""
        return name in self._students
    
    def list_students(self) -> list[str]:
        """Return list of all student names."""
        return list(self._students.keys())
    
    def average_grade(self, name: str) -> float:
        """
        Calculate average grade for a student.
        
        Args:
            name: Student name
        
        Returns:
            Student's overall average
        """
        return self.get_student(name).average_grade()
    
    def get_student_count(self) -> int:
        """Return the number of students in the gradebook."""
        return len(self._students)
    
    def get_class_average(self) -> float:
        """
        Calculate average grade across all students.
        
        Returns:
            Class average, or 0.0 if no students
        """
        if not self._students:
            return 0.0
        
        all_averages = [student.average_grade() for student in self._students.values()]
        return sum(all_averages) / len(all_averages)


# ============================================================================
# USAGE EXAMPLES
# ============================================================================

def main():
    """Demonstrate basic usage of the gradebook system."""
    
    print("=" * 70)
    print("GRADEBOOK SYSTEM DEMO")
    print("=" * 70)
    print()
    
    # Create gradebook and students
    book = Gradebook()
    book.add_student('Albert Einstein')
    book.add_student('Marie Curie')
    
    # Add grades for Einstein
    einstein = book.get_student('Albert Einstein')
    einstein.add_grade(subject_name='Math', score=75, weight=0.05)
    einstein.add_grade(subject_name='Math', score=65, weight=0.15)
    einstein.add_grade(subject_name='Math', score=70, weight=0.80)
    einstein.add_grade(subject_name='Gym', score=100, weight=0.40)
    einstein.add_grade(subject_name='Gym', score=85, weight=0.60)
    
    # Add grades for Curie
    curie = book.get_student('Marie Curie')
    curie.add_grade(subject_name='Physics', score=95, weight=0.30)
    curie.add_grade(subject_name='Physics', score=92, weight=0.70)
    curie.add_grade(subject_name='Chemistry', score=98, weight=1.0)
    
    # Display results
    print(f"Student: {einstein.name}")
    print(f"  Overall average: {einstein.average_grade():.2f}")
    print(f"  Subjects: {', '.join(einstein.list_subjects())}")
    print()
    
    print(f"Student: {curie.name}")
    print(f"  Overall average: {curie.average_grade():.2f}")
    print(f"  Subjects: {', '.join(curie.list_subjects())}")
    print()
    
    print(f"Class average: {book.get_class_average():.2f}")
    print()
    
    # Validate weights
    print("Weight validation:")
    for subject_name, is_valid in einstein.validate_all_subjects().items():
        status = "✓ Valid" if is_valid else "✗ Invalid"
        print(f"  Einstein - {subject_name}: {status}")
    
    for subject_name, is_valid in curie.validate_all_subjects().items():
        status = "✓ Valid" if is_valid else "✗ Invalid"
        print(f"  Curie - {subject_name}: {status}")
    print()
    
    # Display detailed subject information
    print("Detailed subject breakdown:")
    math = einstein.get_subject('Math')
    print(f"\n  Einstein - Math:")
    print(f"    Subject average: {math.weighted_average():.2f}")
    print(f"    Number of grades: {math.get_grade_count()}")
    for i, grade in enumerate(math.grades, 1):
        print(f"    Grade {i}: {grade.score:.1f} (weight: {grade.weight:.2f})")


def error_handling_demo():
    """Demonstrate error handling capabilities."""
    
    print("\n" + "=" * 70)
    print("ERROR HANDLING DEMO")
    print("=" * 70)
    print()
    
    book = Gradebook()
    student = book.add_student('Test Student')
    
    # Test 1: Invalid score
    print("Test 1: Invalid score (out of range)")
    try:
        student.add_grade('Math', score=150, weight=1.0)
    except ValueError as e:
        print(f"  ✓ Caught error: {e}")
    print()
    
    # Test 2: Invalid weight
    print("Test 2: Invalid weight (out of range)")
    try:
        student.add_grade('Math', score=85, weight=1.5)
    except ValueError as e:
        print(f"  ✓ Caught error: {e}")
    print()
    
    # Test 3: Duplicate student
    print("Test 3: Duplicate student")
    try:
        book.add_student('Test Student')
    except ValueError as e:
        print(f"  ✓ Caught error: {e}")
    print()
    
    # Test 4: Non-existent student
    print("Test 4: Non-existent student")
    try:
        book.get_student('Non Existent')
    except KeyError as e:
        print(f"  ✓ Caught error: {e}")
    print()
    
    # Test 5: Non-existent subject
    print("Test 5: Non-existent subject")
    try:
        student.get_subject('Non Existent Subject')
    except KeyError as e:
        print(f"  ✓ Caught error: {e}")
    print()


def fluent_api_demo():
    """Demonstrate fluent API pattern."""
    
    print("=" * 70)
    print("FLUENT API DEMO")
    print("=" * 70)
    print()
    
    book = Gradebook()
    
    # Chain operations
    student = book.add_student('Isaac Newton')
    student.add_grade('Physics', 100, 0.50)
    student.add_grade('Physics', 98, 0.50)
    student.add_grade('Math', 95, 1.0)
    
    print(f"Student: {student.name}")
    print(f"Average: {student.average_grade():.2f}")
    print(f"Subjects: {student.get_subject_count()}")
    print()


# ============================================================================
# PYTHON 3.10+ COMPATIBILITY NOTES
# ============================================================================
"""
This code requires Python 3.10 or higher due to:

1. Built-in generic types (Python 3.9+):
   - list[Grade] instead of typing.List[Grade]
   - dict[str, Subject] instead of typing.Dict[str, Subject]

2. Dataclass slots (Python 3.10+):
   - @dataclass(slots=True) for memory efficiency
   
3. Enhanced type annotations (Python 3.10+):
   - Cleaner union types with | operator (if used)
   - Better type inference

To run this code:
- Python 3.10.0 or higher
- No external dependencies required
- Pure standard library implementation

Backwards compatibility:
- For Python 3.9: Remove slots=True from @dataclass decorators
- For Python 3.8 or lower: Use typing.List and typing.Dict
"""


# ============================================================================
# KEY IMPROVEMENTS OVER ORIGINAL
# ============================================================================
"""
STRUCTURAL IMPROVEMENTS:
✓ Reduced nesting: 3 levels → 2 levels
✓ Named structures: Grade, Subject, Student classes
✓ No defaultdict: Explicit initialization
✓ Modern type hints: Lowercase list/dict (Python 3.9+)
✓ Memory efficient: dataclass slots (Python 3.10+)

ALGORITHMIC IMPROVEMENTS:
✓ Extracted calculations: Each in named method
✓ Single responsibility: One class, one job
✓ No nested loops: Clear delegation
✓ Testable: Each method independent

API IMPROVEMENTS:
✓ Named parameters: Clear intent
✓ Validation: At creation time
✓ Fluent API: Chainable operations
✓ Error handling: Explicit, meaningful messages

MAINTENANCE IMPROVEMENTS:
✓ Unit testable: Each method isolated
✓ Easy debugging: Print objects directly
✓ Extensible: Add fields to dataclasses
✓ Self-documenting: Clear structure
✓ Type-safe: Full type hints with modern syntax
"""


if __name__ == "__main__":
    main()
    error_handling_demo()
    fluent_api_demo()

GRADEBOOK SYSTEM DEMO

Student: Albert Einstein
  Overall average: 80.25
  Subjects: Math, Gym

Student: Marie Curie
  Overall average: 95.45
  Subjects: Physics, Chemistry

Class average: 87.85

Weight validation:
  Einstein - Math: ✓ Valid
  Einstein - Gym: ✓ Valid
  Curie - Physics: ✓ Valid
  Curie - Chemistry: ✓ Valid

Detailed subject breakdown:

  Einstein - Math:
    Subject average: 69.50
    Number of grades: 3
    Grade 1: 75.0 (weight: 0.05)
    Grade 2: 65.0 (weight: 0.15)
    Grade 3: 70.0 (weight: 0.80)

ERROR HANDLING DEMO

Test 1: Invalid score (out of range)
  ✓ Caught error: Score must be 0-100, got 150

Test 2: Invalid weight (out of range)
  ✓ Caught error: Weight must be 0-1, got 1.5

Test 3: Duplicate student
  ✓ Caught error: Student Test Student already exists

Test 4: Non-existent student
  ✓ Caught error: 'Student Non Existent not found'

Test 5: Non-existent subject
  ✓ Caught error: 'Student Test Student has no grades for Non Existent Subject'

FLUENT API DE

### 🚨 When to Refactor to Classes

**Avoid nesting beyond one level:**
- Dictionaries containing dictionaries → Hard to read
- Long tuples → Positional confusion
- Multiple layers of built-in types → Maintenance nightmare

**Time to refactor when:**
1. You're nesting dictionaries inside dictionaries
2. Tuples grow beyond 2 items
3. Code becomes unclear to other programmers

### Refactored Solution: Using Classes

#### Step 1: Create Grade Class Using namedtuple

In [6]:
from collections import namedtuple

# Simple, immutable data container
Grade = namedtuple('Grade', ('score', 'weight'))

# Test it
grade = Grade(score=95, weight=0.45)
print(f"Score: {grade.score}, Weight: {grade.weight}")

# Calculate weighted average
grades = []
grades.append(Grade(95, 0.45))
grades.append(Grade(85, 0.55))

total = sum(score * weight 
            for score, weight in grades)
total_weight = sum(weight for _, weight in grades)
average_grade = total / total_weight

print(f"Weighted average: {average_grade}")

Score: 95, Weight: 0.45
Weighted average: 89.5


#### ⚠️ Limitations of namedtuple

1. **No default arguments** - Can't specify optional parameters
2. **Still accessible by index** - Can lead to bugs in APIs
3. **For many attributes** - Consider `dataclasses` module instead

#### Step 2: Create Subject Class

In [7]:
class Subject:
    def __init__(self):
        self._grades = []
    
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
    
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

# Test it
math = Subject()
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)
print(f"Math average: {math.average_grade()}")

Math average: 69.5


#### Step 3: Create Student Class

In [8]:
class Student:
    def __init__(self):
        self._subjects = defaultdict(Subject)
    
    def get_subject(self, name):
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

# Test it
albert = Student()
math = albert.get_subject('Math')
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)

gym = albert.get_subject('Gym')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)

print(f"Overall average: {albert.average_grade()}")

Overall average: 80.25


#### Step 4: Create Gradebook Class

In [9]:
class Gradebook:
    def __init__(self):
        self._students = defaultdict(Student)
    
    def get_student(self, name):
        return self._students[name]

# Final usage - Much clearer!
book = Gradebook()
albert = book.get_student('Albert Einstein')

math = albert.get_subject('Math')
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)

gym = albert.get_subject('Gym')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)

print(f"Albert's average: {albert.average_grade()}")

Albert's average: 80.25


### 💡 Key Takeaways

✅ **Avoid** dictionaries with nested dictionaries  
✅ **Use** `namedtuple` for lightweight, immutable data  
✅ **Refactor** to multiple classes when internal state gets complicated  
✅ Line count may increase, but **readability** and **maintainability** improve dramatically

### 🎯 Practice Exercise

Try extending the `Grade` class to include a notes field from the teacher:

In [10]:
# Exercise: Extend Grade to support optional notes
# Hint: namedtuple might not be the best choice anymore

# Your code here:


---

## Item 38: Accept Functions Instead of Classes for Simple Interfaces

### Functions as First-Class Objects

Python makes it easy to pass functions as arguments because **functions are first-class objects**. This is ideal for **hooks** and **callbacks**.

### Example 1: Simple Function Hook

In [11]:
# Sorting with a function hook
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']

# EXPLANATION OF TERMS:
# =====================
#
# "Function" - len
# ----------------
# - len is a built-in Python function that returns the length of an object
# - len('Socrates') returns 8 (the number of characters)
# - len('Plato') returns 5
# - In Python, functions are "first-class objects" meaning they can be:
#   * Assigned to variables
#   * Passed as arguments to other functions (like we do here)
#   * Returned from other functions
#
# "Hook" - the key=len part
# -------------------------
# - A "hook" is a function you provide that gets called by another function
# - Also called: callback function, key function, or transform function
# - sort() "hooks into" your function (len) and calls it on each item
# - Think of it as: "Before sorting, transform each item using this function"
#
# How it works:
# -------------
# 1. sort() takes the first name: 'Socrates'
# 2. sort() calls your hook function: len('Socrates') → 8
# 3. sort() takes the second name: 'Archimedes'
# 4. sort() calls your hook function: len('Archimedes') → 10
# 5. sort() does this for all names: 'Plato' → 5, 'Aristotle' → 9
# 6. sort() then sorts based on these numbers: 5, 8, 9, 10
# 7. Result: ['Plato', 'Socrates', 'Aristotle', 'Archimedes']
#
# Without the hook (key=len):
# ---------------------------
# names.sort()  # Would sort alphabetically: ['Archimedes', 'Aristotle', 'Plato', 'Socrates']
#
# With the hook (key=len):
# ------------------------
names.sort(key=len)  # Sorts by length: ['Plato', 'Socrates', 'Aristotle', 'Archimedes']

print(names)

# Visual representation of what happens:
# --------------------------------------
# Original:   ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
#                   ↓            ↓          ↓          ↓
# Hook applied:     8           10          5          9        ← len() called on each
#                   ↓            ↓          ↓          ↓
# Sort by these:    5            8          9         10        ← numbers sorted
#                   ↓            ↓          ↓          ↓
# Final result: ['Plato', 'Socrates', 'Aristotle', 'Archimedes']

['Plato', 'Socrates', 'Aristotle', 'Archimedes']


### Example 2: defaultdict with Function Hook

In [12]:
from collections import defaultdict

def log_missing():
    print('Key added')
    return 0

# Using function as default factory
current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]

result = defaultdict(log_missing, current)
print('Before:', dict(result))

for key, amount in increments:
    result[key] += amount

print('After:', dict(result))

Before: {'green': 12, 'blue': 3}
Key added
Key added
After: {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


### Example 3: Stateful Closure

In [13]:
def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count  # Stateful closure
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
    
    return result, added_count

# Usage
current = {'green': 12, 'blue': 3}
increments = [('red', 5), ('blue', 17), ('orange', 9)]

result, count = increment_with_report(current, increments)
print(f"Result: {dict(result)}")
print(f"Added {count} keys")

Result: {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}
Added 2 keys


### Example 4: Class with State (Better Approach)

In [14]:
class CountMissing:
    def __init__(self):
        self.added = 0
    
    def missing(self):
        self.added += 1
        return 0

# Usage with method reference
counter = CountMissing()
result = defaultdict(counter.missing, current)  # Method reference

for key, amount in increments:
    result[key] += amount

print(f"Added {counter.added} keys")

Added 2 keys


### Example 5: Using `__call__` (Best Approach)

In [15]:
class BetterCountMissing:
    def __init__(self) -> None:
        self.added = 0
    
    def __call__(self):
        self.added += 1
        return 0

# Usage - instance itself is callable
counter = BetterCountMissing()
print(f"Is callable? {callable(counter)}")

result = defaultdict(counter, current)  # Cleaner!
for key, amount in increments:
    result[key] += amount

print(f"Added {counter.added} keys")

Is callable? True
Added 2 keys


In [16]:
"""
Demonstration: Callable Objects in Python
Scenario: Bicycle Shop Pricing System

This example illustrates the progression from simple functions to callable classes,
showing why __call__ provides superior abstraction for stateful operations.
"""

# =============================================================================
# APPROACH 1: Simple Function (Baseline)
# =============================================================================

def calculate_discount_simple(base_price, discount_percent):
    """
    Basic discount calculation using a simple function.
    
    Limitations:
    - No state preservation
    - Discount percentage must be passed every time
    - Cannot encapsulate complex discount logic
    """
    discount_amount = base_price * (discount_percent / 100)
    return base_price - discount_amount


print("=" * 70)
print("APPROACH 1: Simple Function")
print("=" * 70)

bike_price = 500
student_discount = 15
senior_discount = 20

student_price = calculate_discount_simple(bike_price, student_discount)
senior_price = calculate_discount_simple(bike_price, senior_discount)

print(f"Bike Base Price: ${bike_price}")
print(f"Student Price (15% off): ${student_price}")
print(f"Senior Price (20% off): ${senior_price}")
print()


# =============================================================================
# APPROACH 2: Callable Class with __call__ (Optimal)
# =============================================================================

class BikeDiscountCalculator:
    """
    Callable class that encapsulates discount logic with state preservation.
    
    Advantages:
    - Discount rate stored as instance state
    - Object is callable like a function
    - Can track usage statistics (calls made)
    - Extensible for complex discount rules
    """
    
    def __init__(self, discount_percent, customer_type):
        """
        Initialize the discount calculator with fixed discount rate.
        
        Args:
            discount_percent: Percentage discount to apply
            customer_type: Type of customer for this discount
        """
        self.discount_percent = discount_percent
        self.customer_type = customer_type
        self.calculations_made = 0  # Track usage
    
    def __call__(self, base_price):
        """
        Make the instance callable - acts like a function when invoked.
        
        Args:
            base_price: Original price of the bicycle
            
        Returns:
            Final price after applying discount
        """
        self.calculations_made += 1
        discount_amount = base_price * (self.discount_percent / 100)
        final_price = base_price - discount_amount
        return final_price
    
    def get_stats(self):
        """Return usage statistics for this calculator."""
        return f"{self.customer_type} calculator used {self.calculations_made} times"


print("=" * 70)
print("APPROACH 2: Callable Class with __call__")
print("=" * 70)

# Create discount calculators for different customer types
student_calculator = BikeDiscountCalculator(15, "Student")
senior_calculator = BikeDiscountCalculator(20, "Senior")
member_calculator = BikeDiscountCalculator(25, "Member")

# Verify these objects are callable
print(f"Is student_calculator callable? {callable(student_calculator)}")
print(f"Is senior_calculator callable? {callable(senior_calculator)}")
print()

# Use the calculators - they work like functions!
bike_prices = [500, 750, 1000, 1200]

print("Pricing for Multiple Bikes:")
print("-" * 70)

for bike_price in bike_prices:
    student_price = student_calculator(bike_price)  # Calling the object!
    senior_price = senior_calculator(bike_price)
    member_price = member_calculator(bike_price)
    
    print(f"Base Price: ${bike_price:>6} | "
          f"Student: ${student_price:>6.2f} | "
          f"Senior: ${senior_price:>6.2f} | "
          f"Member: ${member_price:>6.2f}")

print()
print("Usage Statistics:")
print(student_calculator.get_stats())
print(senior_calculator.get_stats())
print(member_calculator.get_stats())
print()


# =============================================================================
# APPROACH 3: Advanced - Callable with Complex Logic
# =============================================================================

class AdvancedBikeCalculator:
    """
    Enhanced callable with volume discounts and minimum purchase rules.
    
    Demonstrates why callable classes excel for complex stateful operations.
    """
    
    def __init__(self, base_discount, customer_type, min_purchase=0):
        self.base_discount = base_discount
        self.customer_type = customer_type
        self.min_purchase = min_purchase
        self.total_sales = 0
        self.calculations_made = 0
    
    def __call__(self, base_price, quantity=1):
        """
        Calculate price with volume discounts.
        
        Args:
            base_price: Price per bicycle
            quantity: Number of bikes being purchased
            
        Returns:
            Total final price after all discounts
        """
        self.calculations_made += 1
        
        subtotal = base_price * quantity
        
        # Check minimum purchase requirement
        if subtotal < self.min_purchase:
            print(f" Minimum purchase ${self.min_purchase} not met for {self.customer_type}")
            return subtotal
        
        # Apply base discount
        discount = self.base_discount
        
        # Volume discount logic
        if quantity >= 5:
            discount += 10  # Additional 10% for 5+ bikes
        elif quantity >= 3:
            discount += 5   # Additional 5% for 3+ bikes
        
        # Calculate final price
        discount_amount = subtotal * (discount / 100)
        final_price = subtotal - discount_amount
        
        self.total_sales += final_price
        
        print(f"  {self.customer_type}: {quantity} bikes @ ${base_price} = "
              f"${subtotal} → ${final_price:.2f} ({discount}% off)")
        
        return final_price


print("=" * 70)
print("APPROACH 3: Advanced Callable with Complex Logic")
print("=" * 70)

# Create advanced calculators with minimum purchase requirements
wholesale_calculator = AdvancedBikeCalculator(10, "Wholesale", min_purchase=1000)
corporate_calculator = AdvancedBikeCalculator(15, "Corporate", min_purchase=2000)

print("Volume Purchase Scenarios:")
print("-" * 70)

# Single bike purchase
print("Scenario 1: Single bike ($500)")
wholesale_calculator(500, quantity=1)
print()

# Volume discount kicks in
print("Scenario 2: Three bikes ($500 each)")
wholesale_calculator(500, quantity=3)
print()

print("Scenario 3: Five bikes ($500 each)")
wholesale_calculator(500, quantity=5)
print()

# Corporate with minimum not met
print("Scenario 4: Corporate minimum purchase check")
corporate_calculator(400, quantity=2)  # Only $800, minimum is $2000
print()

print("Scenario 5: Corporate minimum met with volume")
corporate_calculator(500, quantity=5)  # $2500, gets volume + corporate discount
print()

print(f"Total Wholesale Sales: ${wholesale_calculator.total_sales:.2f}")
print(f"Total Corporate Sales: ${corporate_calculator.total_sales:.2f}")
print()


# =============================================================================
# DEMONSTRATION: Using Callables as Hooks
# =============================================================================

class PricingEngine:
    """
    Framework that accepts callable pricing strategies as hooks.
    
    Demonstrates inversion of control using callable objects.
    """
    
    def __init__(self, pricing_strategy):
        """
        Initialize with a callable pricing strategy.
        
        Args:
            pricing_strategy: Any callable that takes (price, quantity) and returns final price
        """
        if not callable(pricing_strategy):
            raise TypeError("Pricing strategy must be callable")
        
        self.pricing_strategy = pricing_strategy
    
    def process_order(self, bike_model, base_price, quantity):
        """Process an order using the configured pricing strategy."""
        print(f"Processing: {quantity}x {bike_model} @ ${base_price} each")
        final_price = self.pricing_strategy(base_price, quantity)
        return final_price


print("=" * 70)
print("DEMONSTRATION: Callables as Hooks")
print("=" * 70)

# Create pricing engine with different strategies
# Note: We use AdvancedBikeCalculator for both since it accepts quantity parameter
student_pricing = AdvancedBikeCalculator(15, "Student", min_purchase=0)
member_pricing = AdvancedBikeCalculator(25, "Premium Member", min_purchase=0)

student_engine = PricingEngine(student_pricing)
member_engine = PricingEngine(member_pricing)

print("Order 1: Student purchasing 1 mountain bike")
student_total = student_engine.process_order("Mountain Bike", 800, 1)
print(f"Final Total: ${student_total:.2f}")
print()

print("Order 2: Premium member purchasing 4 road bikes")
member_total = member_engine.process_order("Road Bike", 1200, 4)
print(f"Final Total: ${member_total:.2f}")
print()


# =============================================================================
# KEY TAKEAWAYS
# =============================================================================

print("=" * 70)
print("KEY INSIGHTS")
print("=" * 70)
print("""
1. CALLABLE OBJECTS via __call__:
   - Act like functions but preserve state
   - Encapsulate complex logic with instance variables
   - Track usage statistics and maintain history
   
2. ADVANTAGES OVER SIMPLE FUNCTIONS:
   - State preservation (discount rates, usage counts)
   - Object-oriented benefits (inheritance, composition)
   - Can be passed as hooks to frameworks
   
3. WHEN TO USE:
   - Need to maintain state between calls
   - Complex initialization logic required
   - Want to track usage or maintain history
   - Building extensible systems with strategy patterns
   
4. THE __call__ METHOD:
   - Makes instance invocable: obj(args) instead of obj.method(args)
   - Creates function-like interface with OOP benefits
   - Enables objects to serve as callbacks/hooks
""")

APPROACH 1: Simple Function
Bike Base Price: $500
Student Price (15% off): $425.0
Senior Price (20% off): $400.0

APPROACH 2: Callable Class with __call__
Is student_calculator callable? True
Is senior_calculator callable? True

Pricing for Multiple Bikes:
----------------------------------------------------------------------
Base Price: $   500 | Student: $425.00 | Senior: $400.00 | Member: $375.00
Base Price: $   750 | Student: $637.50 | Senior: $600.00 | Member: $562.50
Base Price: $  1000 | Student: $850.00 | Senior: $800.00 | Member: $750.00
Base Price: $  1200 | Student: $1020.00 | Senior: $960.00 | Member: $900.00

Usage Statistics:
Student calculator used 4 times
Senior calculator used 4 times
Member calculator used 4 times

APPROACH 3: Advanced Callable with Complex Logic
Volume Purchase Scenarios:
----------------------------------------------------------------------
Scenario 1: Single bike ($500)
 Minimum purchase $1000 not met for Wholesale

Scenario 2: Three bikes ($500 ea

### 📊 Comparison: When to Use What

| Approach | When to Use | Clarity | Statefulness |
|----------|-------------|---------|-------------|
| **Simple Function** | No state needed | ⭐⭐⭐⭐⭐ | ❌ |
| **Closure** | Hidden state | ⭐⭐ | ✅ |
| **Class + Method** | Clear state | ⭐⭐⭐ | ✅ |
| **Class + `__call__`** | Stateful callable | ⭐⭐⭐⭐⭐ | ✅ |

### 💡 Key Takeaways

✅ Functions are **first-class objects** in Python  
✅ Use **simple functions** for stateless hooks  
✅ Use **`__call__`** method for stateful callables  
✅ `__call__` makes class purpose clear as a function-like interface

In [17]:
"""
Example 1: Rate Limiter Decorator Using __call__

Scenario: Limiting API endpoint calls to prevent abuse
Pattern: Stateful decorator that tracks calls and enforces limits
"""

import time
from datetime import datetime, timedelta


class RateLimiter:
    """
    A decorator that limits how many times a function can be called.
    
    Why __call__?
    - Needs to store rate limit configuration (max_calls, time_window)
    - Needs to track call history for each user
    - Needs to act as a function wrapper (callable)
    """
    
    def __init__(self, max_calls, time_window_seconds):
        """
        Initialize the rate limiter.
        
        Args:
            max_calls: Maximum number of calls allowed
            time_window_seconds: Time window in seconds
        """
        self.max_calls = max_calls
        self.time_window = time_window_seconds
        self.call_history = {}  # Tracks calls per user: {user: [timestamp1, timestamp2, ...]}
        self.total_blocked = 0
        self.total_allowed = 0
    
    def __call__(self, func):
        """
        This makes RateLimiter callable - it returns a wrapper function.
        When used as @RateLimiter(...), this method is called.
        
        Args:
            func: The function being decorated
            
        Returns:
            Wrapped function with rate limiting
        """
        def wrapper(user_id, *args, **kwargs):
            current_time = time.time()
            
            # Initialize user's call history if first time
            if user_id not in self.call_history:
                self.call_history[user_id] = []
            
            # Remove old calls outside the time window
            cutoff_time = current_time - self.time_window
            self.call_history[user_id] = [
                call_time for call_time in self.call_history[user_id]
                if call_time > cutoff_time
            ]
            
            # Check if user exceeded rate limit
            if len(self.call_history[user_id]) >= self.max_calls:
                self.total_blocked += 1
                oldest_call = self.call_history[user_id][0]
                wait_time = int(oldest_call + self.time_window - current_time)
                
                print(f"❌ RATE LIMIT EXCEEDED for user '{user_id}'")
                print(f"   Limit: {self.max_calls} calls per {self.time_window}s")
                print(f"   Please wait {wait_time} seconds")
                return None
            
            # Allow the call
            self.call_history[user_id].append(current_time)
            self.total_allowed += 1
            
            print(f"✓ Call allowed for user '{user_id}' "
                  f"({len(self.call_history[user_id])}/{self.max_calls} calls used)")
            
            # Execute the actual function
            return func(user_id, *args, **kwargs)
        
        return wrapper
    
    def get_stats(self):
        """Get statistics about rate limiting."""
        print(f"\n📊 Rate Limiter Statistics:")
        print(f"   Total calls allowed: {self.total_allowed}")
        print(f"   Total calls blocked: {self.total_blocked}")
        print(f"   Active users: {len(self.call_history)}")


# =============================================================================
# Usage Example: Decorating API Endpoints
# =============================================================================

# Create a rate limiter: 3 calls per 10 seconds
api_rate_limiter = RateLimiter(max_calls=3, time_window_seconds=10)


@api_rate_limiter
def get_user_profile(user_id, profile_type="basic"):
    """Simulated API endpoint to get user profile."""
    return f"Profile data for {user_id} (type: {profile_type})"


@api_rate_limiter
def update_user_settings(user_id, setting_name, setting_value):
    """Simulated API endpoint to update user settings."""
    return f"Updated {setting_name}={setting_value} for {user_id}"


# =============================================================================
# Demonstration
# =============================================================================

print("=" * 70)
print("RATE LIMITER DEMONSTRATION")
print("=" * 70)
print(f"Rate Limit: 3 calls per 10 seconds per user\n")

# Scenario 1: Normal usage - user makes 3 calls (all allowed)
print("--- Scenario 1: Normal Usage (3 calls) ---")
result1 = get_user_profile("alice")
print(f"Result: {result1}\n")

result2 = get_user_profile("alice", profile_type="detailed")
print(f"Result: {result2}\n")

result3 = update_user_settings("alice", "theme", "dark")
print(f"Result: {result3}\n")


# Scenario 2: Exceeding rate limit - 4th call is blocked
print("--- Scenario 2: Exceeding Limit (4th call) ---")
result4 = get_user_profile("alice")
print(f"Result: {result4}\n")


# Scenario 3: Different user - has own separate limit
print("--- Scenario 3: Different User (Fresh Limit) ---")
result5 = get_user_profile("bob")
print(f"Result: {result5}\n")

result6 = update_user_settings("bob", "language", "spanish")
print(f"Result: {result6}\n")


# Scenario 4: Wait for time window to expire
print("--- Scenario 4: Waiting for Time Window ---")
print("Waiting 10 seconds for rate limit to reset...")
time.sleep(10)

result7 = get_user_profile("alice")
print(f"Result: {result7}\n")


# Display statistics
api_rate_limiter.get_stats()


# =============================================================================
# Key Insights
# =============================================================================

print("\n" + "=" * 70)
print("WHY __call__ IS PERFECT FOR THIS")
print("=" * 70)
print("""
1. STATEFUL BEHAVIOR:
   - Stores call history for each user (self.call_history)
   - Tracks configuration (max_calls, time_window)
   - Maintains statistics (total_allowed, total_blocked)

2. DECORATOR PATTERN:
   - __call__ receives the function to decorate
   - Returns a wrapper that adds rate limiting logic
   - Original function behavior preserved

3. CLEAN SYNTAX:
   - @api_rate_limiter decorator syntax is elegant
   - Could not achieve this with simple functions
   - State persists across all decorated functions

4. REUSABILITY:
   - One RateLimiter instance can decorate multiple functions
   - All decorated functions share the same rate limit pool
   - Easy to create different limiters for different use cases
""")

print("\n" + "=" * 70)
print("WITHOUT __call__ YOU WOULD NEED:")
print("=" * 70)
print("""
# Awkward alternative without __call__:
limiter = RateLimiter(3, 10)
get_user_profile = limiter.decorate(get_user_profile)  # Ugly!

# Or global state (bad practice):
call_history = {}
def rate_limit_wrapper():  # No way to configure per instance
    ...

The __call__ method makes decorators both stateful AND elegant!
""")

RATE LIMITER DEMONSTRATION
Rate Limit: 3 calls per 10 seconds per user

--- Scenario 1: Normal Usage (3 calls) ---
✓ Call allowed for user 'alice' (1/3 calls used)
Result: Profile data for alice (type: basic)

✓ Call allowed for user 'alice' (2/3 calls used)
Result: Profile data for alice (type: detailed)

✓ Call allowed for user 'alice' (3/3 calls used)
Result: Updated theme=dark for alice

--- Scenario 2: Exceeding Limit (4th call) ---
❌ RATE LIMIT EXCEEDED for user 'alice'
   Limit: 3 calls per 10s
   Please wait 10 seconds
Result: None

--- Scenario 3: Different User (Fresh Limit) ---
✓ Call allowed for user 'bob' (1/3 calls used)
Result: Profile data for bob (type: basic)

✓ Call allowed for user 'bob' (2/3 calls used)
Result: Updated language=spanish for bob

--- Scenario 4: Waiting for Time Window ---
Waiting 10 seconds for rate limit to reset...
✓ Call allowed for user 'alice' (1/3 calls used)
Result: Profile data for alice (type: basic)


📊 Rate Limiter Statistics:
   Total ca

---

## Item 39: Use @classmethod Polymorphism to Construct Objects Generically

### The Problem: Object Construction in Hierarchies

Python only supports **one constructor** (`__init__`) per class. How do we create objects generically across different subclasses?

### Example: MapReduce Implementation

#### Step 1: Define Abstract Base Classes

In [18]:
# Input data abstraction
class InputData:
    def read(self):
        raise NotImplementedError

# Concrete implementation: read from file
class PathInputData(InputData):
    def __init__(self, path):
        super().__init__()
        self.path = path
    
    def read(self):
        with open(self.path) as f:
            return f.read()

In [19]:
# Worker abstraction
class Worker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
    
    def map(self):
        raise NotImplementedError
    
    def reduce(self, other):
        raise NotImplementedError

# Concrete implementation: line counter
class LineCountWorker(Worker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')
    
    def reduce(self, other):
        self.result += other.result

#### Step 2: Manual Construction (Inflexible)

In [20]:
import os

def generate_inputs(data_dir):
    for name in os.listdir(data_dir):
        yield PathInputData(os.path.join(data_dir, name))

def create_workers(input_list):
    workers = []
    for input_data in input_list:
        workers.append(LineCountWorker(input_data))
    return workers

# Problem: Not generic! Hardcoded to PathInputData and LineCountWorker

### ❌ The Problem

If we create new `InputData` or `Worker` subclasses, we must rewrite:
- `generate_inputs()`
- `create_workers()`
- `mapreduce()`

**Not maintainable!**

### ✅ Solution: Class Method Polymorphism

In [21]:
# Generic base class with @classmethod constructor
class GenericInputData:
    def read(self):
        raise NotImplementedError
    
    @classmethod
    def generate_inputs(cls, config):
        raise NotImplementedError

# Concrete implementation
class PathInputData(GenericInputData):
    def __init__(self, path):
        super().__init__()
        self.path = path
    
    def read(self):
        with open(self.path) as f:
            return f.read()
    
    @classmethod
    def generate_inputs(cls, config):
        data_dir = config['data_dir']
        for name in os.listdir(data_dir):
            yield cls(os.path.join(data_dir, name))  # Generic!

In [22]:
# Generic Worker with @classmethod constructor
class GenericWorker:
    def __init__(self, input_data):
        self.input_data = input_data
        self.result = None
    
    def map(self):
        raise NotImplementedError
    
    def reduce(self, other):
        raise NotImplementedError
    
    @classmethod
    def create_workers(cls, input_class, config):
        workers = []
        for input_data in input_class.generate_inputs(config):
            workers.append(cls(input_data))  # Generic!
        return workers

# Concrete implementation
class LineCountWorker(GenericWorker):
    def map(self):
        data = self.input_data.read()
        self.result = data.count('\n')
    
    def reduce(self, other):
        self.result += other.result

In [23]:
# Generic MapReduce function
from threading import Thread

def execute(workers):
    threads = [Thread(target=w.map) for w in workers]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    
    first, *rest = workers
    for worker in rest:
        first.reduce(worker)
    return first.result

def mapreduce(worker_class, input_class, config):
    workers = worker_class.create_workers(input_class, config)
    return execute(workers)

In [24]:
# Test setup
import os
import random

def write_test_files(tmpdir):
    os.makedirs(tmpdir, exist_ok=True)
    for i in range(100):
        with open(os.path.join(tmpdir, str(i)), 'w') as f:
            f.write('\n' * random.randint(0, 100))

tmpdir = 'test_inputs'
write_test_files(tmpdir)

# Now fully generic!
config = {'data_dir': tmpdir}
result = mapreduce(LineCountWorker, PathInputData, config)
print(f'There are {result} lines')

There are 4520 lines


#Example : Class Methods (@classmethod) - Alternative Constructors

# Class Methods vs Inheritance: Comprehensive Comparison

## Executive Summary

**Class methods and inheritance are fundamentally different concepts** that serve distinct purposes in object-oriented programming. This document provides a detailed analysis of their differences, use cases, and relationships.

---

## Table of Contents

1. [Conceptual Differences](#conceptual-differences)
2. [Structural Comparison](#structural-comparison)
3. [Execution Flow Diagrams](#execution-flow-diagrams)
4. [Use Case Analysis](#use-case-analysis)
5. [When to Use What](#when-to-use-what)
6. [Code Examples](#code-examples)

---

## Conceptual Differences

### High-Level Distinction Table

| Aspect | Class Method (`@classmethod`) | Inheritance |
|--------|-------------------------------|-------------|
| **Primary Purpose** | Alternative constructors & factory methods | Code reuse & polymorphism |
| **Relationship Type** | Method of a single class | Relationship between classes |
| **Scope** | Operates on one class | Establishes parent-child hierarchy |
| **What it Creates** | New instances of the same class | New class that extends parent |
| **Access Pattern** | Called on class: `User.from_dict()` | Child inherits: `class Admin(User)` |
| **State** | No instance state, only class state | Inherits all parent attributes/methods |
| **Runtime Behavior** | Receives class as first parameter (`cls`) | Searches method resolution order (MRO) |
| **Flexibility** | Multiple ways to create same class | Specialized versions of base class |

---

## Structural Comparison

### Class Method Architecture

```
┌─────────────────────────────────────┐
│         Single Class: User          │
├─────────────────────────────────────┤
│ Regular Constructor:                │
│   __init__(name, email, password)   │
│                                     │
│ Alternative Constructors:           │
│   @classmethod                      │
│   from_dict(data) → User           │
│                                     │
│   @classmethod                      │
│   from_json(json_str) → User       │
│                                     │
│   @classmethod                      │
│   from_database(id) → User         │
└─────────────────────────────────────┘
        ↓         ↓         ↓
    User obj  User obj  User obj
    (All same class, created differently)
```

### Inheritance Architecture

```
┌─────────────────────────────────────┐
│      Base Class: User               │
│  (Parent/Superclass)                │
├─────────────────────────────────────┤
│ - name                              │
│ - email                             │
│ - password                          │
│ + login()                           │
│ + logout()                          │
└─────────────────────────────────────┘
           ↑           ↑
           │           │
    ┌──────┘           └──────┐
    │                         │
┌───┴────────────┐    ┌───────┴──────────┐
│ Admin          │    │ RegularUser      │
│ (Child Class)  │    │ (Child Class)    │
├────────────────┤    ├──────────────────┤
│ Inherits:      │    │ Inherits:        │
│ - name         │    │ - name           │
│ - email        │    │ - email          │
│ - password     │    │ - password       │
│ - login()      │    │ - login()        │
│ - logout()     │    │ - logout()       │
│                │    │                  │
│ Adds:          │    │ Adds:            │
│ - permissions  │    │ - subscription   │
│ + ban_user()   │    │ + upgrade()      │
└────────────────┘    └──────────────────┘
```

---

## Execution Flow Diagrams

### Class Method Execution Flow

```
User Request: Create user from dictionary
         │
         ▼
    User.from_dict(data)  ◄── Called on CLASS, not instance
         │
         ▼
┌────────────────────────────────────┐
│ @classmethod                       │
│ def from_dict(cls, data):          │
│     # cls = User class             │
│     username = data['username']    │
│     email = data['email']          │
│     password = hash(data['pwd'])   │
│     return cls(username, email,    │
│                password)            │
└────────────────────────────────────┘
         │
         ▼
    User.__init__(...)  ◄── Calls standard constructor
         │
         ▼
   Returns User instance
         │
         ▼
    user_obj = User(username='alice', email='a@ex.com', ...)
```

**Key Points:**
- Single class involved
- `cls` parameter receives the class itself
- Returns instance of same class
- Alternative way to construct objects

### Inheritance Execution Flow

```
Create Admin User Request
         │
         ▼
    admin = Admin(name, email, password)
         │
         ▼
┌────────────────────────────────────┐
│ class Admin(User):  ◄── Inherits   │
│     def __init__(self, name,       │
│                  email, password): │
│         super().__init__(name,     │
│                         email,     │
│                         password)  │
│         self.permissions = []      │
└────────────────────────────────────┘
         │
         ▼
    Calls User.__init__()  ◄── Parent constructor
         │
         ▼
┌────────────────────────────────────┐
│ class User:                        │
│     def __init__(self, name,       │
│                  email, password): │
│         self.name = name           │
│         self.email = email         │
│         self.password = password   │
└────────────────────────────────────┘
         │
         ▼
    Returns to Admin.__init__()
         │
         ▼
    Adds admin-specific attributes
         │
         ▼
    admin_obj = Admin(name='Bob', email='b@ex.com', ...)
                │
                ├─ Has: name, email, password (from User)
                └─ Has: permissions (from Admin)
```

**Key Points:**
- Multiple classes in hierarchy
- Child class extends parent
- `super()` calls parent methods
- Creates specialized instances

---

## Method Resolution Order (MRO) Comparison

### Class Methods: No MRO Needed

```
User.from_dict(data)
    │
    └─► Directly calls User.from_dict()
        (No searching needed - explicit class reference)
```

### Inheritance: Uses MRO

```
admin.login()  ◄── Method call on Admin instance
    │
    ▼
Search Order (MRO):
    1. Admin class  ◄── Check here first
           │ (not found)
           ▼
    2. User class   ◄── Check parent
           │ (found!)
           ▼
       Execute User.login()
```

**MRO Visualization:**

| Class | Method | Found In | Execution Path |
|-------|--------|----------|----------------|
| `admin.login()` | `login()` | User (parent) | Admin → User |
| `admin.ban_user()` | `ban_user()` | Admin (self) | Admin |
| `User.from_dict()` | `from_dict()` | User (self) | Direct call |

---

## Detailed Feature Comparison Table

| Feature | Class Method | Inheritance |
|---------|-------------|-------------|
| **Syntax** | `@classmethod` decorator | `class Child(Parent):` |
| **First Parameter** | `cls` (the class) | `self` (the instance) |
| **Purpose** | Create instances differently | Extend/specialize behavior |
| **Number of Classes** | One class | Multiple classes (hierarchy) |
| **Code Reuse** | Alternative construction | Inherit all parent code |
| **Polymorphism** | No | Yes (method overriding) |
| **`isinstance()` Check** | Always same class | Child is instance of parent too |
| **Memory Footprint** | One class definition | Multiple class definitions |
| **Use Case** | "How to create?" | "What kind of?" |
| **Relationship** | Methods belong to class | Classes relate to each other |
| **Extensibility** | Add more constructors | Add more subclasses |
| **Type Identity** | Same type | Different types (related) |

---

## Use Case Analysis

### When Class Methods Excel

#### Scenario: Multiple Data Sources

```
Problem: Users can register via:
- Web form (username, email, password)
- Social media (OAuth token, provider)
- CSV import (batch data)
- Admin creation (no password initially)

Solution: Class methods for each source

┌─────────────────────────────────┐
│         User Class              │
├─────────────────────────────────┤
│ from_form(form_data)            │ ◄── Web registration
│ from_oauth(token, provider)     │ ◄── Social login
│ from_csv_row(csv_row)           │ ◄── Bulk import
│ from_admin_panel(username)      │ ◄── Admin creation
└─────────────────────────────────┘
```

**Why not inheritance?**
- All create the same type (User)
- No behavioral differences needed
- Just different construction methods

### When Inheritance Excels

#### Scenario: Different User Types with Different Behaviors

```
Problem: System has:
- Regular users (basic features)
- Premium users (extra features)
- Admin users (management features)
- Guest users (limited features)

Solution: Inheritance hierarchy

        User (abstract base)
          │
    ┌─────┼─────┬────────┐
    │     │     │        │
  Guest Regular Premium Admin
    │     │       │      │
    ├─ limited   │      ├─ can ban users
    │   access   │      ├─ view analytics
    │            │      └─ manage content
    │            │
    │            ├─ ad-free
    │            ├─ priority support
    │            └─ advanced features
    │
    └─ read-only
```

**Why not class methods?**
- Different types with different behaviors
- Need polymorphism (same interface, different implementation)
- Need type checking (`isinstance(user, Admin)`)

---

## Identity and Type Checking

### Class Method Type Identity

```python
user1 = User("alice", "a@ex.com", "pass123")
user2 = User.from_dict({'username': 'bob', 'email': 'b@ex.com', 'pwd': 'pass456'})
user3 = User.from_json('{"username": "charlie", ...}')
```

| Check | Result | Reason |
|-------|--------|--------|
| `type(user1) == type(user2)` | `True` | Same class |
| `type(user1) == type(user3)` | `True` | Same class |
| `isinstance(user1, User)` | `True` | Is User |
| `isinstance(user2, User)` | `True` | Is User |
| All three are indistinguishable | `True` | Identical type |

### Inheritance Type Identity

```python
user = User("alice", "a@ex.com", "pass123")
admin = Admin("bob", "b@ex.com", "pass456")
```

| Check | Result | Reason |
|-------|--------|--------|
| `type(user) == type(admin)` | `False` | Different classes |
| `isinstance(user, User)` | `True` | Is User |
| `isinstance(admin, User)` | `True` | Admin inherits from User |
| `isinstance(admin, Admin)` | `True` | Is Admin |
| `isinstance(user, Admin)` | `False` | User is not Admin |

---

## Memory and Performance Comparison

### Class Methods

```
Memory Structure:
┌───────────────────────────┐
│      User Class           │ ◄── One class in memory
│  - __init__               │
│  - from_dict              │
│  - from_json              │
│  - from_csv               │
│  - login                  │
│  - logout                 │
└───────────────────────────┘

Instances:
user1 → {name: 'alice', email: '...', password: '...'} ◄── Same structure
user2 → {name: 'bob', email: '...', password: '...'}   ◄── Same structure
user3 → {name: 'charlie', email: '...', password: '...'} ◄── Same structure
```

### Inheritance

```
Memory Structure:
┌───────────────────────────┐
│      User Class           │ ◄── Base class in memory
│  - __init__               │
│  - login                  │
│  - logout                 │
└───────────────────────────┘
            ▲
            │ inherits
┌───────────┴───────────────┐
│      Admin Class          │ ◄── Additional class in memory
│  - __init__ (extended)    │
│  - ban_user               │
│  - view_analytics         │
└───────────────────────────┘

Instances:
user  → {name: 'alice', ...}                              ◄── User type
admin → {name: 'bob', ..., permissions: [...]}            ◄── Admin type (extended)
```

**Performance Table:**

| Aspect | Class Methods | Inheritance |
|--------|---------------|-------------|
| Class definitions | 1 | 2+ (base + children) |
| Method lookup | Direct | MRO traversal |
| Memory per instance | Same size | Varies by type |
| Runtime overhead | Minimal | Method resolution |
| Instantiation cost | Single constructor path | May call super() chain |

---

## Decision Flow Chart

```
Need to create objects from different data sources?
         │
    ┌────┴────┐
    │   YES   │
    └────┬────┘
         │
         ▼
    All objects are the same type with same behavior?
         │
    ┌────┴────┐
    │   YES   │──────► Use CLASS METHODS
    └─────────┘        (@classmethod)
         │
    ┌────┴────┐
    │   NO    │
    └────┬────┘
         │
         ▼
    Objects need different behaviors/features?
         │
    ┌────┴────┐
    │   YES   │──────► Use INHERITANCE
    └─────────┘        (class Child(Parent))
```

---

## Advanced: Combining Both Patterns

### Pattern: Inheritance WITH Class Methods

```
┌─────────────────────────────────────┐
│         User (Base)                 │
├─────────────────────────────────────┤
│ @classmethod                        │
│ from_dict(data) → User             │
└─────────────────────────────────────┘
            ▲
            │ inherits
┌───────────┴─────────────────────────┐
│         Admin                       │
├─────────────────────────────────────┤
│ Inherits: from_dict()               │ ◄── Also gets class method!
│                                     │
│ @classmethod                        │
│ from_dict_with_permissions(data)    │ ◄── Can add own class methods
└─────────────────────────────────────┘
```

**Usage:**

```python
# Using inherited class method
admin1 = Admin.from_dict(data)  # Returns Admin instance!

# Using Admin-specific class method
admin2 = Admin.from_dict_with_permissions(data, perms)
```

**Flow Diagram:**

```
Admin.from_dict(data)  ◄── Call class method on child class
    │
    ▼
User.from_dict(cls, data)  ◄── Inherited from User
    │
    ├─ cls parameter = Admin (not User!)
    │
    ▼
return cls(username, email, password)
    │
    ▼
Admin(username, email, password)  ◄── Creates Admin instance
```

**Key Insight:** When inherited, `cls` parameter in class methods refers to the **calling class**, not the defining class.

---

## Comparison Summary Table

### Quick Reference Guide

| Situation | Use This | Example |
|-----------|----------|---------|
| Multiple ways to create **same type** | Class methods | `User.from_json()`, `User.from_dict()` |
| Different **types** with shared behavior | Inheritance | `class Admin(User)`, `class Guest(User)` |
| Parse different data formats | Class methods | `Date.from_string()`, `Date.from_timestamp()` |
| Specialize behavior for subtypes | Inheritance | `Admin` has `ban_user()`, `User` doesn't |
| Factory pattern | Class methods | `Config.from_file()`, `Config.from_env()` |
| "IS-A" relationship | Inheritance | Admin IS-A User |
| Alternative constructors | Class methods | Multiple ways to build same thing |
| Polymorphism needed | Inheritance | Different types, same interface |
| Single class, multiple creation paths | Class methods | ✓ |
| Multiple classes, shared code | Inheritance | ✓ |

---

## Real-World Analogies

### Class Methods Analogy

**Building a Car:**
```
Car Factory
│
├─ build_from_parts()      ◄── Assemble from components
├─ build_from_blueprint()  ◄── Construct from plans
├─ build_from_salvage()    ◄── Rebuild from old car
└─ build_custom()          ◄── Custom order

Result: All produce the same type (Car)
```

### Inheritance Analogy

**Vehicle Types:**
```
Vehicle (base)
│
├─ Car        ◄── Four wheels, passenger transport
├─ Truck      ◄── Cargo transport, different behavior
├─ Motorcycle ◄── Two wheels, different controls
└─ Bus        ◄── Public transport, many seats

Result: Different types with different behaviors
```

---

## Key Takeaways

### Class Methods
- ✓ Alternative constructors for the **same class**
- ✓ Multiple ways to create objects
- ✓ Factory methods
- ✓ Single class involved
- ✗ No polymorphism
- ✗ Not for creating different types

### Inheritance
- ✓ Creating **related but different** classes
- ✓ Code reuse through parent-child relationship
- ✓ Polymorphism support
- ✓ Type specialization
- ✗ Not for alternative construction
- ✗ Creates class hierarchy

### Bottom Line

**Class methods answer:** "How do I create this?"

**Inheritance answers:** "What kind of thing is this?"

They are complementary patterns that solve different problems and can be used together effectively.

In [25]:
"""
Example 2: Class Methods (@classmethod) - Alternative Constructors

Scenario: User registration system with multiple ways to create users
Pattern: Using @classmethod to provide different construction methods
"""

from __future__ import annotations
from datetime import datetime
from typing import Literal


AccountType = Literal["regular", "premium", "admin"]


class User:
    """
    User account with multiple ways to create instances.
    
    Why @classmethod?
    - Provides alternative constructors for different input formats
    - Can validate/transform data before creating instance
    - Returns proper instance of the class (works with inheritance)
    """
    
    def __init__(
        self,
        username: str,
        email: str,
        password_hash: str,
        account_type: AccountType = "regular"
    ) -> None:
        """
        Standard constructor - requires all parameters.
        
        Args:
            username: User's username
            email: User's email address
            password_hash: Hashed password
            account_type: Type of account (regular, premium, admin)
        """
        self.username: str = username
        self.email: str = email
        self.password_hash: str = password_hash
        self.account_type: AccountType = account_type
        self.created_at: datetime = datetime.now()
        self.login_count: int = 0
    
    @classmethod
    def from_registration_form(cls, form_data: dict[str, str]) -> User:
        """
        Create user from web registration form data.
        
        This is an ALTERNATIVE CONSTRUCTOR using @classmethod.
        
        Args:
            form_data: Dictionary with form fields
            
        Returns:
            New User instance
        """
        print(f" Creating user from registration form...")
        
        # Extract and validate form data
        username = form_data['username']
        email = form_data['email']
        password = form_data['password']
        
        # Hash the password (simplified for demo)
        password_hash = f"hash_{password}_secure"
        
        # Return new instance using standard constructor
        return cls(username, email, password_hash, account_type="regular")
    
    @classmethod
    def from_social_login(
        cls,
        provider: str,
        social_id: str,
        social_email: str,
        display_name: str
    ) -> User:
        """
        Create user from social media login (Facebook, Google, etc.).
        
        Another ALTERNATIVE CONSTRUCTOR using @classmethod.
        
        Args:
            provider: Social media provider (facebook, google)
            social_id: User's ID from social provider
            social_email: Email from social provider
            display_name: Display name from social provider
            
        Returns:
            New User instance
        """
        print(f"🌐 Creating user from {provider} login...")
        
        # Generate username from social data
        username = f"{display_name.lower().replace(' ', '_')}_{provider}"
        
        # Social logins don't have passwords - use social ID
        password_hash = f"social_{provider}_{social_id}"
        
        # Return new instance
        return cls(username, social_email, password_hash, account_type="regular")
    
    @classmethod
    def from_admin_panel(
        cls,
        username: str,
        email: str,
        is_admin: bool = False
    ) -> User:
        """
        Create user from admin panel (no password needed initially).
        
        Yet another ALTERNATIVE CONSTRUCTOR using @classmethod.
        
        Args:
            username: User's username
            email: User's email
            is_admin: Whether user should have admin privileges
            
        Returns:
            New User instance
        """
        print(f"👑 Creating user from admin panel...")
        
        # Generate temporary password
        password_hash = "temp_password_change_required"
        
        # Set account type based on admin flag
        account_type: AccountType = "admin" if is_admin else "regular"
        
        # Return new instance
        return cls(username, email, password_hash, account_type)
    
    def display_info(self) -> None:
        """Display user information."""
        print(f"\n{'=' * 60}")
        print(f"👤 User: {self.username}")
        print(f"📧 Email: {self.email}")
        print(f"🔐 Password Hash: {self.password_hash}")
        print(f"⭐ Account Type: {self.account_type}")
        print(f"📅 Created: {self.created_at.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"{'=' * 60}")


# =============================================================================
# Demonstration of Different Construction Methods
# =============================================================================

print("=" * 70)
print("CLASS METHOD DEMONSTRATION - Alternative Constructors")
print("=" * 70)
print()

# Method 1: Standard constructor (normal __init__)
print("--- Method 1: Standard Constructor ---")
user1 = User(
    username="john_doe",
    email="john@example.com",
    password_hash="hash_abc123_secure"
)
user1.display_info()
print()


# Method 2: From registration form (using @classmethod)
print("--- Method 2: From Registration Form ---")
registration_data = {
    'username': 'alice_smith',
    'email': 'alice@example.com',
    'password': 'mySecurePassword123'
}
user2 = User.from_registration_form(registration_data)
user2.display_info()
print()


# Method 3: From social login (using @classmethod)
print("--- Method 3: From Social Media Login ---")
user3 = User.from_social_login(
    provider="google",
    social_id="1234567890",
    social_email="bob.wilson@gmail.com",
    display_name="Bob Wilson"
)
user3.display_info()
print()


# Method 4: From admin panel (using @classmethod)
print("--- Method 4: From Admin Panel ---")
user4 = User.from_admin_panel(
    username="admin_sarah",
    email="sarah@company.com",
    is_admin=True
)
user4.display_info()
print()


# =============================================================================
# Another Example: Date Parsing
# =============================================================================

print("\n" + "=" * 70)
print("BONUS EXAMPLE: Date Parsing with @classmethod")
print("=" * 70)
print()


class Date:
    """Simple date class with multiple constructors."""
    
    def __init__(self, year: int, month: int, day: int) -> None:
        self.year: int = year
        self.month: int = month
        self.day: int = day
    
    @classmethod
    def from_string(cls, date_string: str) -> Date:
        """Create Date from string like '2025-10-31'."""
        print(f" Parsing date string: {date_string}")
        year, month, day = map(int, date_string.split('-'))
        return cls(year, month, day)
    
    @classmethod
    def from_timestamp(cls, timestamp: float) -> Date:
        """Create Date from Unix timestamp."""
        print(f" Converting timestamp: {timestamp}")
        dt = datetime.fromtimestamp(timestamp)
        return cls(dt.year, dt.month, dt.day)
    
    @classmethod
    def today(cls) -> Date:
        """Create Date for today."""
        print(f"📆 Getting today's date")
        now = datetime.now()
        return cls(now.year, now.month, now.day)
    
    def __str__(self) -> str:
        return f"{self.year:04d}-{self.month:02d}-{self.day:02d}"


# Different ways to create Date objects
date1 = Date(2025, 10, 31)  # Standard constructor
print(f"Standard constructor: {date1}\n")

date2 = Date.from_string("2024-12-25")  # From string
print(f"From string: {date2}\n")

date3 = Date.from_timestamp(1609459200)  # From timestamp
print(f"From timestamp: {date3}\n")

date4 = Date.today()  # Today
print(f"Today: {date4}\n")


# =============================================================================
# Key Insights
# =============================================================================

print("\n" + "=" * 70)
print("WHY @classmethod IS USEFUL")
print("=" * 70)
print("""
1. ALTERNATIVE CONSTRUCTORS:
   - One class, multiple ways to create instances
   - Each @classmethod is a named constructor
   - More descriptive than __init__ with complex logic

2. RECEIVES THE CLASS (cls):
   - First parameter is the class itself (not instance)
   - Can call cls(...) to create new instance
   - Works correctly with inheritance

3. CLEANER CODE:
   - User.from_registration_form(data) is clear
   - User.from_social_login(...) explains its purpose
   - Better than: User(parse_form_data(data))

4. COMMON PATTERNS:
   - Parsing different input formats
   - Factory methods
   - Configuration-based object creation
   - Singleton patterns
""")

print("\n" + "=" * 70)
print("@classmethod vs REGULAR METHOD")
print("=" * 70)
print("""
Regular Method:
    def method(self):        # Receives instance
        self.attribute = 5   # Works with instance data
    
    usage: user.method()     # Called on instance

Class Method:
    @classmethod
    def method(cls):         # Receives class
        return cls(...)      # Creates new instance
    
    usage: User.method()     # Called on class itself

Static Method:
    @staticmethod
    def method():            # Receives neither
        return "helper"      # Just a utility function
    
    usage: User.method()     # Called on class
""")

CLASS METHOD DEMONSTRATION - Alternative Constructors

--- Method 1: Standard Constructor ---

👤 User: john_doe
📧 Email: john@example.com
🔐 Password Hash: hash_abc123_secure
⭐ Account Type: regular
📅 Created: 2025-10-31 17:07:22

--- Method 2: From Registration Form ---
 Creating user from registration form...

👤 User: alice_smith
📧 Email: alice@example.com
🔐 Password Hash: hash_mySecurePassword123_secure
⭐ Account Type: regular
📅 Created: 2025-10-31 17:07:22

--- Method 3: From Social Media Login ---
🌐 Creating user from google login...

👤 User: bob_wilson_google
📧 Email: bob.wilson@gmail.com
🔐 Password Hash: social_google_1234567890
⭐ Account Type: regular
📅 Created: 2025-10-31 17:07:22

--- Method 4: From Admin Panel ---
👑 Creating user from admin panel...

👤 User: admin_sarah
📧 Email: sarah@company.com
🔐 Password Hash: temp_password_change_required
⭐ Account Type: admin
📅 Created: 2025-10-31 17:07:22


BONUS EXAMPLE: Date Parsing with @classmethod

Standard constructor: 2025-10-31


# Why Use Multiple Class Methods Instead of One `__init__`?

## Core Concept

Instead of creating one massive constructor that accepts every possible parameter combination, use separate class methods where each one handles a specific way of creating objects.

---

## The Problem: Mega Constructor Anti-Pattern

### What Goes Wrong

When you try to handle all possible construction scenarios in a single `__init__` method, you create a constructor that accepts numerous optional parameters. This leads to a deeply nested chain of conditional logic that checks which parameters were provided.

### Problems with This Approach

| Problem | Impact | Why It Matters |
|---------|--------|----------------|
| **Confusing Interface** | Users don't know which parameters to use | Must read documentation to understand valid parameter combinations |
| **No Parameter Validation** | Can pass multiple conflicting sources accidentally | No way to enforce mutual exclusivity at the type level |
| **Poor Error Messages** | Generic "invalid parameters" doesn't help | Hard to diagnose what went wrong when construction fails |
| **Maintenance Nightmare** | Adding new source requires changing one giant method | Must modify existing, tested code with each addition |
| **Testing Complexity** | Must test all parameter combinations | Exponential growth in test cases |
| **Type Hints Are Lies** | Everything marked Optional but isn't really | Type checker can't validate correct usage |
| **Poor IDE Support** | Auto-complete shows irrelevant parameters | Overwhelming parameter list confuses developers |
| **No Code Reuse** | Logic duplicated in each branch | Similar parsing logic repeated multiple times |
| **Hard to Document** | One docstring for multiple behaviors | Documentation becomes complex and hard to maintain |
| **Violates Single Responsibility** | One method does too many things | Constructor becomes parser + loader + validator + builder |

---

## The Solution: Multiple Class Methods

### How It Works

Create a simple constructor that accepts only the core required parameters. Then create separate class methods for each alternative construction pattern. Each class method handles its specific data source and eventually calls the standard constructor.

### Benefits of This Approach

| Benefit | Impact | Why It Matters |
|---------|--------|----------------|
| **Clear Intent** | Method name explains what it does | Self-documenting code |
| **Focused Parameters** | Each method has exactly what it needs | No confusing optional parameters |
| **Type Safety** | Type hints are accurate | Type checker can validate properly |
| **Easy to Test** | Test each method independently | Simple, isolated test cases |
| **Excellent IDE Support** | Auto-complete shows relevant parameters only | No clutter or confusion |
| **Code Reuse** | Methods can call each other | Eliminate duplication |
| **Easy to Extend** | Add new methods without touching old code | Open-closed principle |
| **Self-Documenting** | Each method has focused docstring | Easy to understand and maintain |
| **Better Error Messages** | Specific to the construction method | Clear what went wrong |
| **Single Responsibility** | Each method does ONE thing | Easy to maintain and debug |

---

## Conceptual Comparison

### Single Constructor Approach

**Structure:**
- One constructor accepts all possible parameters
- Long list of optional parameters
- Complex conditional logic to determine which parameters were provided
- Each branch handles different data source
- Must validate parameter combinations manually

**Problems:**
- Unclear which parameters are needed
- Easy to make mistakes
- Hard to understand what's happening
- Difficult to modify safely
- Type hints don't help

### Multiple Class Methods Approach

**Structure:**
- Simple constructor accepts only required parameters
- Separate class method for each data source
- Each method has focused, clear parameters
- Methods can call each other to reuse logic
- Standard constructor does the actual object creation

**Benefits:**
- Crystal clear what each method does
- Impossible to make parameter combination mistakes
- Easy to understand each method
- Safe to add new methods
- Type hints are accurate and helpful

---

## Additional Benefits Table

| Aspect | Single `__init__` | Multiple Class Methods |
|--------|-------------------|------------------------|
| **Auto-complete Support** | Shows ALL parameters (confusing) | Shows only relevant parameters |
| **Type Hints** | Complex Optional unions | Simple, specific types |
| **Documentation** | One giant docstring | Each method self-documented |
| **Code Reuse** | Duplicated logic in branches | Methods can call each other |
| **Adding New Sources** | Modify existing `__init__` | Add new class method (no changes to existing) |
| **Error Messages** | Generic "invalid parameters" | Specific to construction method |
| **IDE Support** | Poor (too many options) | Excellent (clear methods) |
| **Testing** | Complex (all combinations) | Simple (one method at a time) |
| **Maintenance** | Risky (change affects everything) | Safe (isolated changes) |
| **Readability** | Deeply nested if-elif-else | Clean, separate methods |

---

## Real-World Impact

### Adding a New Construction Method

#### Single Constructor Approach

**What You Must Do:**
- Add new optional parameter to existing constructor signature
- Add new branch to existing if-elif chain
- Implement parsing logic in the branch
- Risk breaking existing functionality
- Update all tests to account for new parameter
- Update documentation for increasingly complex constructor

**Risks:**
- Modified existing, tested code
- Increased complexity of already-complex method
- Potential to break existing functionality
- Must retest all existing scenarios
- Maintenance burden increases

#### Multiple Class Methods Approach

**What You Must Do:**
- Add new class method
- Implement focused logic in the new method
- Test only the new method
- Existing code remains untouched

**Benefits:**
- No existing code modified
- Zero risk to existing functionality
- Simple, focused implementation
- Only new method needs testing
- Self-documenting addition
- Can reuse existing helper methods

---

## Type Hints Impact

### Single Constructor Type Hints

**The Problem:**
Everything becomes optional because the constructor accepts multiple different parameter combinations. Type hints must mark all parameters as optional, even though in reality specific combinations are required.

**Why This Fails:**
- Type hints claim everything is optional
- Reality is more complex (must provide specific combinations)
- Type checker can't validate correct usage
- Developers must read documentation to understand requirements
- IDE can't provide meaningful hints or warnings

### Multiple Class Methods Type Hints

**The Solution:**
Each method has specific, required parameters. Type hints accurately represent what's needed for each construction method.

**Why This Succeeds:**
- Type hints accurately represent requirements
- Type checker can validate calls properly
- IDE provides helpful warnings for incorrect usage
- Self-documenting code
- Catch errors at development time, not runtime

---

## Performance Consideration

### Memory Impact

**Question:** Don't multiple methods use more memory?

**Answer:** The impact is negligible while maintainability gains are massive.

| Aspect | Single `__init__` | Multiple Class Methods |
|--------|-------------------|------------------------|
| **Memory per class** | 1 method definition | ~5 method definitions (tiny overhead) |
| **Memory per instance** | Identical | Identical |
| **Execution speed** | Complex branching (slower) | Direct path (faster) |
| **Development speed** | Slow (complex logic) | Fast (simple logic) |
| **Debugging time** | High (complex branches) | Low (isolated methods) |
| **Maintenance cost** | High (risky changes) | Low (safe changes) |

**Conclusion:** The tiny memory cost of additional method definitions is vastly outweighed by the massive gains in code clarity, maintainability, and developer productivity.

---

## Design Principles

### Single Responsibility Principle

**Definition:** Each function should do ONE thing well.

**How Mega-Constructor Violates It:**
A single constructor that handles multiple data sources is doing many different things:
- Constructing from parameters
- Parsing JSON
- Querying databases
- Handling OAuth
- Parsing CSV files
- Parsing XML
- Extracting from dictionaries
- Validating parameter combinations

**How Class Methods Follow It:**
Each method has a single, clear responsibility:
- Standard constructor: create object from validated parameters
- from_json: parse JSON and create object
- from_database: load from database and create object
- from_oauth: handle OAuth and create object

### Open-Closed Principle

**Definition:** Open for extension, closed for modification.

**How Mega-Constructor Violates It:**
Adding new construction method requires:
- Modifying existing constructor signature
- Modifying existing conditional logic
- Risk breaking existing, tested code
- Retesting all existing scenarios

**How Class Methods Follow It:**
Adding new construction method requires:
- Adding new method (no modification to existing code)
- No risk to existing functionality
- Only test new method
- Existing code remains unchanged and stable

---

## Practical Examples of Benefits

### Scenario: Multiple Data Sources

**Context:** Users can be created from web forms, social media logins, CSV imports, database loads, or admin panels.

**Single Constructor Problem:**
- One constructor with 10+ optional parameters
- Complex validation logic
- Unclear which parameters to use
- Easy to make mistakes
- Hard to understand

**Class Methods Solution:**
- from_form: handles web form data
- from_social_login: handles OAuth
- from_csv_row: handles CSV import
- from_database: handles database load
- from_admin_panel: handles admin creation

Each is clear, focused, and impossible to misuse.

### Scenario: Adding New Feature

**Context:** Need to add YAML file support for user creation.

**Single Constructor Problem:**
- Must modify existing constructor
- Add yaml_string parameter to long list
- Add another branch to if-elif chain
- Risk breaking existing branches
- Must retest everything

**Class Methods Solution:**
- Add from_yaml method
- Implement focused logic
- Can reuse from_dict helper
- Zero impact on existing code
- Only test new method

---

## IDE and Tooling Impact

### Auto-complete Experience

**Single Constructor:**
When typing the constructor call, IDE shows every possible parameter. Developer must:
- Scroll through long parameter list
- Read each parameter name
- Guess which ones are needed together
- Check documentation to understand valid combinations

**Class Methods:**
When typing class name, IDE shows all available methods. Developer:
- Sees clear method names (from_json, from_database, etc.)
- Selects appropriate method for their use case
- Sees only relevant parameters for that method
- Gets immediate clarity about what's needed

### Error Detection

**Single Constructor:**
- Type checker can't validate parameter combinations
- Errors caught at runtime
- Generic error messages
- Must debug to understand issue

**Class Methods:**
- Type checker validates each method call
- Errors caught at development time
- Specific error messages
- Immediate clarity about problem

---

## Documentation Impact

### Single Constructor Documentation

**Challenge:**
Must document:
- All possible parameter combinations
- Which parameters work together
- Which parameters are mutually exclusive
- Complex conditional logic
- Multiple examples for each scenario

**Result:**
- Long, complex docstring
- Hard to find relevant information
- Overwhelming for users
- Difficult to maintain

### Multiple Class Methods Documentation

**Advantage:**
Each method documents:
- Its single, clear purpose
- Its specific parameters
- One focused example
- One clear use case

**Result:**
- Short, focused docstrings
- Easy to find relevant information
- Clear and understandable
- Simple to maintain

---

## Summary

### The Core Answer

**Why not put everything in `__init__`?**

Because it creates:
- Confusing interfaces that are hard to use correctly
- Complex, unmaintainable code with nested conditionals
- Poor type safety where type hints don't match reality
- Difficult testing with exponential test cases
- Bad error messages that don't help debugging
- Risky changes where modifications affect everything

**Why use multiple class methods?**

Because they provide:
- Clear, focused interfaces that are obvious to use
- Simple, maintainable code with single responsibilities
- Strong type safety where hints accurately represent requirements
- Easy testing with isolated test cases
- Helpful error messages specific to each method
- Safe extensibility where changes don't affect existing code

### Bottom Line

**Each method should do ONE thing well.**

Don't make `__init__` into a Swiss Army knife that tries to handle every possible scenario. Instead, use class methods to provide focused, clear, maintainable alternative constructors. Each constructor method should have a single, obvious purpose.

The small overhead of additional method definitions is vastly outweighed by the enormous gains in code clarity, maintainability, type safety, and developer productivity.

---

## Key Principles to Remember

1. **Single Responsibility:** Each method does one thing well
2. **Open-Closed:** Open for extension (add methods), closed for modification (don't change existing)
3. **Clear Intent:** Method names should be self-documenting
4. **Type Safety:** Type hints should accurately represent requirements
5. **Easy Testing:** Each method should be testable in isolation
6. **Maintainability:** Changes should be safe and localized
7. **Developer Experience:** Code should be obvious and hard to misuse

**Remember: Clear is better than clever!**

### 💡 Key Takeaways

✅ Python only supports **one `__init__`** per class  
✅ Use **`@classmethod`** to define alternative constructors  
✅ Class method polymorphism enables **generic object construction**  
✅ Eliminates need to rewrite glue code for new subclasses

---

## Item 40: Initialize Parent Classes with super

### The Old Way (Direct Call) - ❌ Problematic

In [26]:
class MyBaseClass:
    def __init__(self, value):
        self.value = value

class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)  # Direct call - works but brittle

child = MyChildClass()
print(f"Value: {child.value}")

Value: 5


### Problem 1: Multiple Inheritance Order Confusion

# Problem: Multiple Inheritance Order Confusion

## The Issue

When using multiple inheritance in Python, **the order of parent classes matters**. Different orderings can produce the same final result, which creates confusion and makes code behavior unclear.

---

## What's Happening in the Code

### Class Definitions

**TimesTwo class:**
- Multiplies `self.value` by 2
- Sets `self.value *= 2` in its `__init__`

**PlusFive class:**
- Adds 5 to `self.value`
- Sets `self.value += 5` in its `__init__`

---

## The Problem Demonstrated

### Case 1: OneWay Class

**Parent Order:** `MyBaseClass, TimesTwo, PlusFive`

```
class OneWay(MyBaseClass, TimesTwo, PlusFive):
```

**What happens when creating `OneWay(5)`:**

1. `MyBaseClass.__init__(self, 5)` → sets `value = 5`
2. `TimesTwo.__init__(self)` → `value *= 2` → `value = 10`
3. `PlusFive.__init__(self)` → `value += 5` → `value = 15`

**Result:** `foo.value = 15`

---

### Case 2: AnotherWay Class

**Parent Order:** `MyBaseClass, PlusFive, TimesTwo`

```
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
```

**What happens when creating `AnotherWay(5)`:**

1. `MyBaseClass.__init__(self, 5)` → sets `value = 5`
2. `TimesTwo.__init__(self)` → `value *= 2` → `value = 10`  ← Same order as OneWay!
3. `PlusFive.__init__(self)` → `value += 5` → `value = 15`  ← Same order as OneWay!

**Result:** `bar.value = 15`

---

## Why This Is Confusing

### The Core Problem

| Aspect | Issue | Impact |
|--------|-------|--------|
| **Different Class Definitions** | Parent order is different | Should produce different behavior |
| **Same Execution Order** | Both call parents in same sequence | Produces identical results |
| **Method Resolution Order (MRO)** | MRO can be non-intuitive | Hard to predict behavior |
| **Misleading Code** | Class declaration order suggests one thing | Actual execution does another |
| **Maintenance Nightmare** | Changing parent order might not change behavior | Confusing for developers |

---

## Why They Execute in the Same Order

### Method Resolution Order (MRO)

Python uses **C3 Linearization** algorithm to determine the order in which parent classes are searched for methods.

**OneWay MRO:**
```
OneWay → MyBaseClass → TimesTwo → PlusFive → object
```

**AnotherWay MRO:**
```
AnotherWay → MyBaseClass → TimesTwo → PlusFive → object
```

**Key Point:** Despite different parent order in class definition, the MRO ends up being the same due to how C3 linearization resolves the inheritance graph.

---

## Problems This Creates

### 1. **Unpredictable Behavior**

**What developers expect:**
- Different parent order = different behavior
- `OneWay` should execute differently than `AnotherWay`

**What actually happens:**
- Both execute in the same order
- Identical results despite different declarations

### 2. **Hidden Dependencies**

| Problem | Description |
|---------|-------------|
| **Order Dependency** | Behavior depends on MRO, not obvious declaration order |
| **Complex Rules** | MRO algorithm is complex and non-intuitive |
| **Hard to Debug** | Must understand MRO to predict behavior |
| **Fragile Code** | Small changes can have unexpected effects |

### 3. **Maintenance Issues**

**Scenarios that cause problems:**
- Developer changes parent order expecting different behavior
- Behavior doesn't change as expected
- Code works "by accident" due to MRO
- Hard to understand why code behaves certain way

### 4. **Testing Challenges**

**Issues:**
- Must test MRO behavior explicitly
- Can't rely on declared parent order
- Edge cases are hard to predict
- Behavior might change with Python version updates

---

## The Real Danger

### Why This Is Bad Design

**The Confusion Matrix:**

| What You Write | What You Think Happens | What Actually Happens |
|---------------|------------------------|---------------------|
| `class OneWay(Base, A, B)` | Calls A then B | Calls A then B (correct) |
| `class AnotherWay(Base, B, A)` | Calls B then A | Calls A then B (surprise!) |

**The Problem:**
You can't tell from the class declaration what will actually happen. You must compute or check the MRO.

---

## Deeper Analysis: Why MRO Matters

### The Method Resolution Order Algorithm

**C3 Linearization ensures:**
1. Children come before parents
2. Parent order is preserved when possible
3. No class appears before its parents
4. Each class appears only once

**In this example:**
Both `OneWay` and `AnotherWay` have the same effective dependency graph, so C3 linearization produces the same MRO regardless of the declared order in certain cases.

---

## Practical Problems

### Problem Table

| Issue | Description | Impact |
|-------|-------------|--------|
| **Code Clarity** | Can't understand behavior from reading code | Must check MRO explicitly |
| **Maintenance** | Changing order might not work as expected | Wasted debugging time |
| **Team Confusion** | Different developers have different mental models | Bugs and misunderstandings |
| **Fragility** | Code works for wrong reasons | Breaks unexpectedly later |
| **Testing Gap** | Unclear what to test | Bugs slip through |

---

## Why This Pattern Should Be Avoided

### Fundamental Issues with Multiple Inheritance

**When multiple inheritance causes problems:**

1. **Diamond Problem:** Multiple paths to same base class
2. **Order Confusion:** Declared order ≠ execution order
3. **Method Name Conflicts:** Which parent's method wins?
4. **State Conflicts:** Which parent's state is used?
5. **Implicit Dependencies:** Hidden coupling between parents

### Specific Issues in This Example

| Issue | Why It's Bad |
|-------|-------------|
| **Side Effects in `__init__`** | Parent constructors modify state | Creates order dependencies |
| **Stateful Operations** | `*= 2` and `+= 5` depend on execution order | Order-sensitive behavior |
| **No Clear Responsibility** | Mixing multiplication and addition in inheritance | Violates single responsibility |
| **Implicit Coupling** | Parents implicitly coupled through shared state | Fragile design |

---

## Better Design Alternatives

### Why This Is An Anti-Pattern

**The fundamental problem:** Using inheritance for behavior composition when composition would be clearer.

### Composition Over Inheritance

**Instead of:**
```
Multiple inheritance with order-dependent side effects
```

**Use:**
```
Single class that explicitly composes behaviors
```

### Strategy Pattern

**Instead of:**
```
Inheritance for different calculation orders
```

**Use:**
```
Strategy objects that encapsulate calculation logic
```

### Explicit Calculation

**Instead of:**
```
Hidden order-dependent operations in __init__
```

**Use:**
```
Explicit methods that show what happens when
```

---

## Key Takeaways

### The Core Problems

1. **Different declarations, same behavior** → Confusing
2. **MRO is not obvious from code** → Hard to predict
3. **Order-dependent side effects** → Fragile
4. **Implicit state dependencies** → Coupled
5. **Hard to maintain** → Technical debt

### What Makes This Bad

| Bad Practice | Why It's Wrong |
|-------------|----------------|
| **Multiple inheritance for behavior** | Should use composition |
| **Side effects in `__init__`** | Creates order dependencies |
| **Implicit coupling** | Parents share state without clear contract |
| **Non-obvious behavior** | Can't predict from reading code |
| **Order-dependent logic** | Fragile and confusing |

---

## The Bottom Line

### Summary of the Problem

**The Issue:**
Two classes with different parent orderings produce the same result because the Method Resolution Order (MRO) happens to be identical. This creates confusion because:

- The declared order suggests different behavior
- The actual behavior is the same
- Developers can't predict this without understanding MRO
- It works "by accident" rather than by design

**Why This Is Bad:**
- Code behavior is non-obvious
- Maintenance is difficult
- Easy to introduce bugs
- Violates principle of least surprise
- Creates hidden dependencies

**The Solution:**
Avoid this pattern entirely. Use composition, strategy pattern, or explicit method calls instead of relying on multiple inheritance with order-dependent side effects.

---

## Warning Signs

### When Your Code Has This Problem

You know you have this issue when:

- ✗ Multiple inheritance with more than 2 parents
- ✗ Parent `__init__` methods have side effects
- ✗ Order of parents affects behavior
- ✗ Can't predict behavior without checking MRO
- ✗ Different orderings produce unexpected results
- ✗ Behavior depends on implicit state sharing

**If you see these patterns, refactor to use composition instead.**

---

## Recommended Approach

Instead of using multiple inheritance with order-dependent side effects:

1. **Use composition** - contain objects rather than inherit
2. **Use strategy pattern** - encapsulate algorithms
3. **Use explicit methods** - make operations obvious
4. **Document MRO** - if must use multiple inheritance, document expected MRO
5. **Minimize side effects** - avoid stateful operations in `__init__`
6. **Single responsibility** - each class does one thing

**Remember: Explicit is better than implicit!**

In [27]:
class TimesTwo:
    def __init__(self):
        self.value *= 2

class PlusFive:
    def __init__(self):
        self.value += 5

# Parent order: MyBaseClass, TimesTwo, PlusFive
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

foo = OneWay(5)
print(f'First ordering value is (5 * 2) + 5 = {foo.value}')

# Different parent order but same __init__ calls!
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)  # Same order as OneWay
        PlusFive.__init__(self)

bar = AnotherWay(5)
print(f'Second ordering value is {bar.value}')  # Same result - confusing!

First ordering value is (5 * 2) + 5 = 15
Second ordering value is 15


### Problem 2: Diamond Inheritance

In [28]:
# Diamond inheritance pattern
#       MyBaseClass
#      /           \
# TimesSeven    PlusNine
#      \           /
#       ThisWay

class TimesSeven(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 7

class PlusNine(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 9

class ThisWay(TimesSeven, PlusNine):
    def __init__(self, value):
        TimesSeven.__init__(self, value)
        PlusNine.__init__(self, value)  # Resets value!

foo = ThisWay(5)
print(f'Should be (5 * 7) + 9 = 44 but is {foo.value}')  # Wrong!

Should be (5 * 7) + 9 = 44 but is 14


### ✅ Solution: Use `super()`

In [29]:
class TimesSevenCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)  # Python handles MRO
        self.value *= 7

class PlusNineCorrect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value += 9

class GoodWay(TimesSevenCorrect, PlusNineCorrect):
    def __init__(self, value):
        super().__init__(value)

foo = GoodWay(5)
print(f'Should be 7 * (5 + 9) = 98 and is {foo.value}')  # Correct!

Should be 7 * (5 + 9) = 98 and is 98


### Understanding Method Resolution Order (MRO)

In [30]:
# Python uses C3 linearization algorithm
mro_str = '\n'.join(repr(cls) for cls in GoodWay.mro())
print("Method Resolution Order:")
print(mro_str)

# Order of execution:
# 1. GoodWay.__init__
# 2. TimesSevenCorrect.__init__
# 3. PlusNineCorrect.__init__
# 4. MyBaseClass.__init__
# Then work happens in reverse order!

Method Resolution Order:
<class '__main__.GoodWay'>
<class '__main__.TimesSevenCorrect'>
<class '__main__.PlusNineCorrect'>
<class '__main__.MyBaseClass'>
<class 'object'>


### Different Ways to Call `super()`

In [31]:
# All three are equivalent:

class ExplicitTrisect(MyBaseClass):
    def __init__(self, value):
        super(ExplicitTrisect, self).__init__(value)
        self.value /= 3

class AutomaticTrisect(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value)
        self.value /= 3

class ImplicitTrisect(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)  # ⭐ Preferred!
        self.value /= 3

# All produce the same result
assert ExplicitTrisect(9).value == 3
assert AutomaticTrisect(9).value == 3
assert ImplicitTrisect(9).value == 3
print("All three methods work!")

All three methods work!


In [32]:
"""
Simple Example: Understanding super() and MRO

Key Takeaways Demonstrated:
1. Always use super() instead of direct __init__ calls
2. Python's MRO (C3 linearization) handles diamond inheritance
3. Prefer super() with zero arguments (most maintainable)
4. Only provide parameters to super() when wrapping/reusing specific functionality
"""

from __future__ import annotations


# =============================================================================
# Example 1: Basic super() Usage (Key Takeaway #1)
# =============================================================================

print("=" * 70)
print("EXAMPLE 1: Why Use super() Instead of Direct Calls")
print("=" * 70)


class Animal:
    def __init__(self, name: str) -> None:
        print(f"Animal.__init__ called for {name}")
        self.name = name


# ❌ BAD: Direct parent call
class BadDog(Animal):
    def __init__(self, name: str, breed: str) -> None:
        Animal.__init__(self, name)  # ❌ Direct call - not flexible
        self.breed = breed


# ✅ GOOD: Using super()
class GoodDog(Animal):
    def __init__(self, name: str, breed: str) -> None:
        super().__init__(name)  # ✅ Uses super() - flexible and maintainable
        self.breed = breed


print("\nBad way (direct call):")
bad_dog = BadDog("Rex", "Labrador")

print("\nGood way (super()):")
good_dog = GoodDog("Max", "Golden Retriever")



EXAMPLE 1: Why Use super() Instead of Direct Calls

Bad way (direct call):
Animal.__init__ called for Rex

Good way (super()):
Animal.__init__ called for Max


In [33]:

# =============================================================================
# Example 2: Diamond Inheritance and MRO (Key Takeaways #2 & #3)
# =============================================================================

print("\n" + "=" * 70)
print("EXAMPLE 2: Diamond Inheritance - MRO Magic")
print("=" * 70)


class Employee:
    """Base class for all employees."""
    
    def __init__(self, name: str, employee_id: str) -> None:
        print(f"  Employee.__init__ called for {name}")
        self.name = name
        self.employee_id = employee_id


class Manager(Employee):
    """Manages a team."""
    
    def __init__(self, name: str, employee_id: str, team_size: int, **kwargs) -> None:
        print(f"  Manager.__init__ called for {name}")
        super().__init__(name, employee_id, **kwargs)  # ✅ Pass along extra kwargs
        self.team_size = team_size


class Engineer(Employee):
    """Technical role."""
    
    def __init__(self, name: str, employee_id: str, programming_language: str, **kwargs) -> None:
        print(f"  Engineer.__init__ called for {name}")
        super().__init__(name, employee_id, **kwargs)  # ✅ Pass along extra kwargs
        self.programming_language = programming_language


# Diamond inheritance: EngineeringManager inherits from both Manager and Engineer
class EngineeringManager(Manager, Engineer):
    """Manages engineering team - combines both roles."""
    
    def __init__(
        self,
        name: str,
        employee_id: str,
        team_size: int,
        programming_language: str
    ) -> None:
        print(f"  EngineeringManager.__init__ called for {name}")
        # ✅ super() handles the diamond - calls next in MRO
        # Pass programming_language via kwargs for Engineer class
        super().__init__(
            name, 
            employee_id, 
            team_size,
            programming_language=programming_language
        )


print("\nCreating EngineeringManager:")
eng_manager = EngineeringManager("Alice", "ENG001", 5, "Python")

print("\nMRO (Method Resolution Order):")
for i, cls in enumerate(EngineeringManager.__mro__, 1):
    print(f"  {i}. {cls.__name__}")

print("\nFinal object attributes:")
print(f"  Name: {eng_manager.name}")
print(f"  Employee ID: {eng_manager.employee_id}")
print(f"  Team Size: {eng_manager.team_size}")
print(f"  Programming Language: {eng_manager.programming_language}")




EXAMPLE 2: Diamond Inheritance - MRO Magic

Creating EngineeringManager:
  EngineeringManager.__init__ called for Alice
  Manager.__init__ called for Alice
  Engineer.__init__ called for Alice
  Employee.__init__ called for Alice

MRO (Method Resolution Order):
  1. EngineeringManager
  2. Manager
  3. Engineer
  4. Employee
  5. object

Final object attributes:
  Name: Alice
  Employee ID: ENG001
  Team Size: 5
  Programming Language: Python


In [34]:

# =============================================================================
# Example 3: When to Pass Parameters to super() (Key Takeaway #4)
# =============================================================================

print("\n" + "=" * 70)
print("EXAMPLE 3: When to Pass Parameters to super()")
print("=" * 70)


class Logger:
    """Base logging functionality."""
    
    def __init__(self, log_level: str = "INFO") -> None:
        print(f"  Logger.__init__ with level: {log_level}")
        self.log_level = log_level
    
    def log(self, message: str) -> None:
        print(f"[{self.log_level}] {message}")


class FileLogger(Logger):
    """Logs to a file with custom log level."""
    
    def __init__(self, filename: str, log_level: str = "INFO") -> None:
        print(f"  FileLogger.__init__ for file: {filename}")
        #  Pass parameters when wrapping/reusing specific functionality
        super().__init__(log_level)  # Explicitly pass log_level
        self.filename = filename
    
    def log(self, message: str) -> None:
        print(f"  [Writing to {self.filename}]")
        super().log(message)


class TimestampedLogger(Logger):
    """Adds timestamp to logs."""
    
    def __init__(self, log_level: str = "INFO") -> None:
        print(f"  TimestampedLogger.__init__")
        # super() with zero arguments when just passing through
        super().__init__(log_level)
        self.include_timestamp = True
    
    def log(self, message: str) -> None:
        from datetime import datetime
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        super().log(f"[{timestamp}] {message}")


print("\nFileLogger example (passing parameters):")
file_logger = FileLogger("app.log", "DEBUG")
file_logger.log("Application started")

print("\nTimestampedLogger example (zero arguments):")
ts_logger = TimestampedLogger("WARNING")
ts_logger.log("System warning")


# =============================================================================
# Example 4: Comparison - Why super() Matters
# =============================================================================

print("\n" + "=" * 70)
print("EXAMPLE 4: The Problem Without super()")
print("=" * 70)


class Vehicle:
    def __init__(self, brand: str) -> None:
        print(f"  Vehicle.__init__ for {brand}")
        self.brand = brand
        self.init_count = getattr(self, 'init_count', 0) + 1


class Car(Vehicle):
    def __init__(self, brand: str, model: str) -> None:
        print(f"  Car.__init__")
        Vehicle.__init__(self, brand)  # Direct call for comparison
        self.model = model


class Electric(Vehicle):
    def __init__(self, brand: str, battery_size: int) -> None:
        print(f"  Electric.__init__")
        Vehicle.__init__(self, brand)  # Direct call for comparison
        self.battery_size = battery_size


# ❌ BAD: Without super() - calls parent multiple times
class BrokenElectricCar(Car, Electric):
    def __init__(self, brand: str, model: str, battery_size: int) -> None:
        print(f"  BrokenElectricCar.__init__")
        # ❌ Direct calls - Vehicle.__init__ gets called TWICE!
        Car.__init__(self, brand, model)
        Electric.__init__(self, brand, battery_size)


print("\n❌ Broken version (without proper super()):")
broken = BrokenElectricCar("Tesla", "Model 3", 75)
print(f"  Vehicle.__init__ was called {broken.init_count} times! ❌")


# Now let's fix it with super()
class BetterCar(Vehicle):
    def __init__(self, brand: str, model: str, **kwargs) -> None:
        print(f"  BetterCar.__init__")
        super().__init__(brand, **kwargs)  # ✅ Pass along kwargs
        self.model = model


class BetterElectric(Vehicle):
    def __init__(self, brand: str, battery_size: int, **kwargs) -> None:
        print(f"  BetterElectric.__init__")
        super().__init__(brand, **kwargs)  # ✅ Pass along kwargs
        self.battery_size = battery_size


# ✅ GOOD: With super() throughout
class WorkingElectricCar(BetterCar, BetterElectric):
    def __init__(self, brand: str, model: str, battery_size: int) -> None:
        print(f"  WorkingElectricCar.__init__")
        # ✅ super() ensures each __init__ called exactly once
        super().__init__(brand=brand, model=model, battery_size=battery_size)


print("\n✅ Working version (with super()):")
working = WorkingElectricCar("Tesla", "Model 3", 75)
print(f"  Vehicle.__init__ was called {working.init_count} time! ✅")


# =============================================================================
# Key Insights Summary
# =============================================================================

print("\n" + "=" * 70)
print("KEY INSIGHTS SUMMARY")
print("=" * 70)
print("""
1. ALWAYS USE super():
   ✅ super().__init__(...)
   ❌ ParentClass.__init__(self, ...)
   
2. MRO HANDLES DIAMOND INHERITANCE:
   - Python automatically resolves complex inheritance
   - Each class called exactly once
   - Order determined by C3 linearization
   
3. PREFER super() WITH ZERO ARGUMENTS:
   - Most maintainable
   - Automatically works with MRO
   - Example: super().__init__()
   
4. PASS PARAMETERS ONLY WHEN NEEDED:
   - When wrapping specific functionality
   - When customizing parent behavior
   - Example: super().__init__(custom_param)
   
WHY IT MATTERS:
- Prevents bugs in complex inheritance
- Makes code more maintainable
- Enables proper diamond inheritance
- Reduces coupling between classes
""")


EXAMPLE 3: When to Pass Parameters to super()

FileLogger example (passing parameters):
  FileLogger.__init__ for file: app.log
  Logger.__init__ with level: DEBUG
  [Writing to app.log]
[DEBUG] Application started

TimestampedLogger example (zero arguments):
  TimestampedLogger.__init__
  Logger.__init__ with level: WARNING
[WARNING] [2025-10-31 17:07:22] System warning

EXAMPLE 4: The Problem Without super()

❌ Broken version (without proper super()):
  BrokenElectricCar.__init__
  Car.__init__
  Vehicle.__init__ for Tesla
  Electric.__init__
  Vehicle.__init__ for Tesla
  Vehicle.__init__ was called 2 times! ❌

✅ Working version (with super()):
  WorkingElectricCar.__init__
  BetterCar.__init__
  BetterElectric.__init__
  Vehicle.__init__ for Tesla
  Vehicle.__init__ was called 1 time! ✅

KEY INSIGHTS SUMMARY

1. ALWAYS USE super():
   ✅ super().__init__(...)
   ❌ ParentClass.__init__(self, ...)
   
2. MRO HANDLES DIAMOND INHERITANCE:
   - Python automatically resolves complex inhe

### 💡 Key Takeaways

✅ Always use **`super()`** instead of direct `__init__` calls  
✅ Python's **MRO** (C3 linearization) handles diamond inheritance  
✅ Prefer **`super()`** with zero arguments (most maintainable)  
✅ Only provide parameters to `super()` when wrapping/reusing specific functionality

---

## Item 41: Consider Composing Functionality with Mix-in Classes

### What is a Mix-in?

A **mix-in** is a class that:
- Defines a **small set of additional methods**
- Has **no instance attributes**
- Doesn't require **`__init__`** to be called
- Provides **generic functionality** that can be applied to many classes

# Mix-in Pattern: ToDictMixin

## What Is This?

A **mix-in class** that adds dictionary conversion functionality to any class that inherits from it.

---

## Core Concept

**Mix-in:** A class that provides specific functionality to be "mixed into" other classes through inheritance.

**Purpose:** Give any class the ability to convert itself to a dictionary representation.

---

## How It Works

### The Mix-in Class

```
ToDictMixin provides:
  - to_dict() method → Converts object to dictionary
  - _traverse_dict() → Handles dictionary traversal
  - _traverse() → Handles different data types (dict, list, objects, etc.)
```

### The Logic

**When you call `to_dict()` on an object:**

1. **Check if value is ToDictMixin** → Call its `to_dict()` method
2. **Check if value is dict** → Recursively traverse the dictionary
3. **Check if value is list** → Traverse each item in the list
4. **Check if value has `__dict__`** → It's an object, traverse its attributes
5. **Otherwise** → Return the value as-is (primitive type)

---

## Example Usage

### BinaryTree with Mix-in

```
class BinaryTree(ToDictMixin):
    - Has: value, left, right attributes
    - Gets: to_dict() method automatically
```

**What happens:**
```
tree = BinaryTree(10, left=..., right=...)
tree.to_dict()  → Converts entire tree structure to dictionary
```

---

## Why Use Mix-ins?

| Benefit | Description |
|---------|-------------|
| **Reusability** | Add same functionality to multiple classes |
| **Single Responsibility** | Mix-in does ONE thing well |
| **No Code Duplication** | Write once, use everywhere |
| **Composability** | Combine multiple mix-ins |
| **Clean Separation** | Functionality separated from data structure |

---

## Key Characteristics

### What Makes This a Mix-in

1. **Provides utility functionality** (not core business logic)
2. **Designed to be inherited** (not used standalone)
3. **No `__init__` method** (doesn't initialize state)
4. **Works with any class** (generic, reusable)
5. **Single purpose** (just dictionary conversion)

---

## The Pattern in Action

### Without Mix-in (Bad)

```
Every class needs to implement to_dict():
  - BinaryTree implements to_dict()
  - LinkedList implements to_dict()
  - Graph implements to_dict()
  - ... (code duplication everywhere!)
```

### With Mix-in (Good)

```
Write once:
  - ToDictMixin implements to_dict()

Use everywhere:
  - class BinaryTree(ToDictMixin): ...
  - class LinkedList(ToDictMixin): ...
  - class Graph(ToDictMixin): ...
```

---

## Simple Analogy

**Mix-in is like a power-up in a video game:**
- You inherit the mix-in → You gain its abilities
- BinaryTree gains `to_dict()` ability
- Any class can gain this ability by inheriting from ToDictMixin

---

## Key Takeaway

**Mix-in Pattern = Reusable functionality through inheritance**

- **What:** Provides specific feature (dictionary conversion)
- **How:** Through inheritance (`class MyClass(ToDictMixin)`)
- **Why:** Avoid code duplication, maintain single responsibility
- **When:** When multiple classes need the same utility functionality

---

## Common Mix-in Examples

| Mix-in Type | Purpose |
|------------|---------|
| **ToDictMixin** | Convert objects to dictionaries |
| **JSONSerializableMixin** | Add JSON serialization |
| **ComparableMixin** | Add comparison operators |
| **TimestampMixin** | Add created_at/updated_at timestamps |
| **LoggingMixin** | Add logging capabilities |

**Bottom Line:** Mix-ins are small, focused classes that add specific functionality to other classes through inheritance. They promote code reuse and separation of concerns.

### Example 1: ToDictMixin

In [35]:
class ToDictMixin:
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    
    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output
    
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value

In [36]:
# Use the mix-in
class BinaryTree(ToDictMixin):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

# Create a tree
tree = BinaryTree(10,
    left=BinaryTree(7, right=BinaryTree(9)),
    right=BinaryTree(13, left=BinaryTree(11)))

# Convert to dict automatically!
import json
print(json.dumps(tree.to_dict(), indent=2))

{
  "value": 10,
  "left": {
    "value": 7,
    "left": null,
    "right": {
      "value": 9,
      "left": null,
      "right": null
    }
  },
  "right": {
    "value": 13,
    "left": {
      "value": 11,
      "left": null,
      "right": null
    },
    "right": null
  }
}


### Overriding Mix-in Behavior

In [37]:
# Tree with parent reference (would cause cycles)
class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None, right=None, parent=None):
        super().__init__(value, left=left, right=right)
        self.parent = parent
    
    def _traverse(self, key, value):
        if isinstance(value, BinaryTreeWithParent) and key == 'parent':
            return value.value  # Prevent cycles
        else:
            return super()._traverse(key, value)

# Test it
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)

print(json.dumps(root.to_dict(), indent=2))

{
  "value": 10,
  "left": {
    "value": 7,
    "left": null,
    "right": {
      "value": 9,
      "left": null,
      "right": null,
      "parent": 7
    },
    "parent": 10
  },
  "right": null,
  "parent": null
}


### Example 2: Composable Mix-ins

In [38]:
import json

class JsonMixin:
    @classmethod
    def from_json(cls, data):
        kwargs = json.loads(data)
        return cls(**kwargs)
    
    def to_json(self):
        return json.dumps(self.to_dict())

# Compose multiple mix-ins
class DatacenterRack(ToDictMixin, JsonMixin):
    def __init__(self, switch=None, machines=None):
        self.switch = Switch(**switch)
        self.machines = [Machine(**kwargs) for kwargs in machines]

class Switch(ToDictMixin, JsonMixin):
    def __init__(self, ports=None, speed=None):
        self.ports = ports
        self.speed = speed

class Machine(ToDictMixin, JsonMixin):
    def __init__(self, cores=None, ram=None, disk=None):
        self.cores = cores
        self.ram = ram
        self.disk = disk

In [39]:
# Test serialization
serialized = """{
    "switch": {"ports": 5, "speed": 1e9},
    "machines": [
        {"cores": 8, "ram": 32e9, "disk": 5e12},
        {"cores": 4, "ram": 16e9, "disk": 1e12}
    ]
}"""

deserialized = DatacenterRack.from_json(serialized)
roundtrip = deserialized.to_json()

print("Original and roundtrip match:",
      json.loads(serialized) == json.loads(roundtrip))

Original and roundtrip match: True


### 💡 Key Takeaways

✅ Mix-ins avoid multiple inheritance complexity  
✅ No instance attributes or `__init__` required  
✅ Can be **composed** to build complex functionality  
✅ Can be **overridden** for custom behavior  
✅ Works with both **instance methods** and **class methods**

---

## Item 42: Prefer Public Attributes Over Private Ones

### Python's Visibility Types

Python has only **two** types of attribute visibility:
1. **Public**: `self.public_field`
2. **Private**: `self.__private_field`

In [40]:
class MyObject:
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10
    
    def get_private_field(self):
        return self.__private_field

foo = MyObject()
print(f"Public: {foo.public_field}")
print(f"Private (via method): {foo.get_private_field()}")

# Direct access to private fails
try:
    print(foo.__private_field)
except AttributeError as e:
    print(f"Error: {e}")

Public: 5
Private (via method): 10
Error: 'MyObject' object has no attribute '__private_field'


### How Private Attributes Actually Work

In [41]:
# Python transforms private attribute names
print("Object's __dict__:")
print(foo.__dict__)

# You can still access it!
print(f"\nAccessing 'private' field: {foo._MyObject__private_field}")

Object's __dict__:
{'public_field': 5, '_MyObject__private_field': 10}

Accessing 'private' field: 10


### Problem with Private Attributes in Subclasses

In [42]:
class MyParentObject:
    def __init__(self):
        self.__private_field = 71

class MyChildObject(MyParentObject):
    def get_private_field(self):
        return self.__private_field  # Won't work!

baz = MyChildObject()
try:
    baz.get_private_field()
except AttributeError as e:
    print(f"Error: {e}")

Error: 'MyChildObject' object has no attribute '_MyChildObject__private_field'


### The Right Way: Protected Attributes

In [43]:
class MyStringClass:
    def __init__(self, value):
        # Protected by convention (single underscore)
        self._value = value
    
    def get_value(self):
        return str(self._value)

class MyIntegerSubclass(MyStringClass):
    def get_value(self):
        return int(self._value)  # Works!

foo = MyIntegerSubclass(5)
print(f"Integer value: {foo.get_value()}")

Integer value: 5


### When to Use Private Attributes

**Only** use private attributes (`__`) to avoid **naming conflicts** in public APIs:

In [44]:
# Public API class (many unknown subclasses)
class ApiClass:
    def __init__(self):
        self.__value = 5  # Avoid conflicts
    
    def get(self):
        return self.__value

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'  # No conflict!

a = Child()
print(f"{a.get()} and {a._value} are different")

5 and hello are different


###  Naming Conventions Summary

| Style | Usage | Meaning |
|-------|-------|----------|
| `public_field` | Normal use |  Accessible to everyone |
| `_protected_field` | Internal API |  Use with caution |
| `__private_field` | Rare |  Avoid naming conflicts only |

###  Key Takeaways

**Document** protected fields instead of forcing privacy  
 Use **single underscore** (`_protected`) for internal APIs  
 **Rarely** use double underscore (only for public API naming conflicts)  
 Remember: **"We are all consenting adults here"**

---

## Item 43: Inherit from collections.abc for Custom Container Types

### The Problem with Custom Containers

In [45]:
# Simple custom list with frequency counting
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)
    
    def frequency(self):
        counts = {}
        for item in self:
            counts[item] = counts.get(item, 0) + 1
        return counts

# Works great!
foo = FrequencyList(['a', 'b', 'a', 'c', 'b', 'a', 'd'])
print(f"Length: {len(foo)}")
foo.pop()
print(f"After pop: {foo}")
print(f"Frequency: {foo.frequency()}")

Length: 7
After pop: ['a', 'b', 'a', 'c', 'b', 'a']
Frequency: {'a': 3, 'b': 2, 'c': 1}


### Custom Sequence Without Inheriting from list

In [46]:
# Binary tree node
class BinaryNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

# Make it indexable
class IndexableNode(BinaryNode):
    def _traverse(self):
        if self.left is not None:
            yield from self.left._traverse()
        yield self
        if self.right is not None:
            yield from self.right._traverse()
    
    def __getitem__(self, index):
        for i, item in enumerate(self._traverse()):
            if i == index:
                return item.value
        raise IndexError(f'Index {index} is out of range')

In [47]:
# Build a tree
tree = IndexableNode(
    10,
    left=IndexableNode(
        5,
        left=IndexableNode(2),
        right=IndexableNode(6, right=IndexableNode(7))
    ),
    right=IndexableNode(15, left=IndexableNode(11))
)

# Can index it!
print(f"Index 0: {tree[0]}")
print(f"Index 1: {tree[1]}")
print(f"11 in tree? {11 in tree}")
print(f"Tree as list: {list(tree)}")

# But len() doesn't work!
try:
    print(len(tree))
except TypeError as e:
    print(f"Error: {e}")

Index 0: 2
Index 1: 5
11 in tree? True
Tree as list: [2, 5, 6, 7, 10, 11, 15]
Error: object of type 'IndexableNode' has no len()


### Adding `__len__` Method

In [48]:
class SequenceNode(IndexableNode):
    def __len__(self):
        for count, _ in enumerate(self._traverse(), 1):
            pass
        return count

tree = SequenceNode(
    10,
    left=SequenceNode(
        5,
        left=SequenceNode(2),
        right=SequenceNode(6, right=SequenceNode(7))
    ),
    right=SequenceNode(15, left=SequenceNode(11))
)

print(f"Tree length: {len(tree)}")

# But still missing count() and index()!
try:
    tree.count(10)
except AttributeError as e:
    print(f"Missing method: {e}")

Tree length: 7
Missing method: 'SequenceNode' object has no attribute 'count'


### ✅ Solution: Use collections.abc

In [49]:
from collections.abc import Sequence

# Validates required methods
class BadType(Sequence):
    pass

try:
    foo = BadType()
except TypeError as e:
    print(f"Error: {e}")

Error: Can't instantiate abstract class BadType with abstract methods __getitem__, __len__


In [50]:
# Provides free methods!
class BetterNode(SequenceNode, Sequence):
    pass

tree = BetterNode(
    10,
    left=BetterNode(
        5,
        left=BetterNode(2),
        right=BetterNode(6, right=BetterNode(7))
    ),
    right=BetterNode(15, left=BetterNode(11))
)

# Now we get index() and count() for free!
print(f"Index of 7: {tree.index(7)}")
print(f"Count of 10: {tree.count(10)}")

Index of 7: 3
Count of 10: 1


###  Common collections.abc Classes

| Abstract Base Class | Required Methods | Free Methods |
|---------------------|------------------|---------------|
| **Sequence** | `__getitem__`, `__len__` | `index`, `count`, `__contains__`, `__iter__`, `__reversed__` |
| **MutableSequence** | + `__setitem__`, `__delitem__`, `insert` | + `append`, `reverse`, `extend`, `pop`, `remove` |
| **Set** | `__contains__`, `__iter__`, `__len__` | `<=`, `<`, `==`, `!=`, `>`, `>=`, `&`, `\|`, `-`, `^` |
| **MutableSet** | + `add`, `discard` | + `clear`, `pop`, `remove`, `__ior__`, `__iand__`, `__ixor__`, `__isub__` |
| **Mapping** | `__getitem__`, `__iter__`, `__len__` | `get`, `keys`, `items`, `values`, `__contains__` |
| **MutableMapping** | + `__setitem__`, `__delitem__` | + `pop`, `popitem`, `clear`, `update`, `setdefault` |

###  Key Takeaways

 inherit from **`list` or `dict`** for simple use cases  
 Use **`collections.abc`** for custom container types  
 Automatic **validation** of required methods  
 Get many **free methods** automatically  
 Ensures **correct interface** and behaviors

# Python Collections Abstract Base Classes - Complete Examples Guide

This notebook provides comprehensive examples for all Python Collections ABCs:
1. Sequence
2. MutableSequence
3. Set
4. MutableSet
5. Mapping
6. MutableMapping

---
## 1. SEQUENCE

### Overview
A Sequence is an ordered collection that supports indexing and has a defined length.

### Required Methods

#### `__getitem__(index)` - Retrieves an element at a specific position

In [51]:
class SimpleSequence:
    def __init__(self, items):
        self._items = list(items)
    
    def __getitem__(self, index):
        return self._items[index]
    
    def __len__(self):
        return len(self._items)

# Usage
seq = SimpleSequence([10, 20, 30, 40])
print(seq[0])  # Output: 10
print(seq[2])  # Output: 30

10
30


#### `__len__()` - Returns the number of elements in the sequence

In [52]:
class NameSequence:
    def __init__(self, names):
        self._names = tuple(names)
    
    def __getitem__(self, index):
        return self._names[index]
    
    def __len__(self):
        return len(self._names)

# Usage
names = NameSequence(['Alice', 'Bob', 'Charlie'])
print(len(names))  # Output: 3

3


### Free Methods (Automatically Available)

#### `index(value)` - Finds the position of the first occurrence of a value

In [58]:
# CORRECTED CODE - SimpleSequence with proper inheritance

from collections.abc import Sequence

class SimpleSequence(Sequence):  # Must inherit from Sequence!
    def __init__(self, items):
        self._items = list(items)
    
    # REQUIRED: Implement these two methods
    def __getitem__(self, index):
        return self._items[index]
    
    def __len__(self):
        return len(self._items)
    
    # That's it! Now you automatically get:
    # - index()
    # - count()
    # - __contains__() (for 'in' operator)
    # - __iter__() (for loops)
    # - __reversed__() (for reversed())


# Now it works!
seq = SimpleSequence(['apple', 'banana', 'cherry', 'banana'])
print(seq.index('banana'))  # Output: 1 ✓
print(seq.count('banana'))  # Output: 2 ✓
print('apple' in seq)        # Output: True ✓

# Iteration works too
for item in seq:
    print(item, end=' ')
# Output: apple banana cherry banana ✓

1
2
True
apple banana cherry banana 

#### `count(value)` - Counts how many times a value appears

In [59]:
seq = SimpleSequence([1, 2, 3, 2, 2, 4])
print(seq.count(2))  # Output: 3

3


#### `__contains__(value)` - Checks if a value exists (enables `in` operator)

In [61]:
seq = SimpleSequence([5, 10, 15, 20])
print(10 in seq)   # Output: True
print(100 in seq)  # Output: False

True
False


#### `__iter__()` - Enables iteration over the sequence

In [62]:
seq = SimpleSequence(['red', 'green', 'blue'])
for color in seq:
    print(color)
# Output:
# red
# green
# blue

red
green
blue


#### `__reversed__()` - Enables reverse iteration

In [63]:
seq = SimpleSequence([1, 2, 3, 4, 5])
for num in reversed(seq):
    print(num, end=' ')
# Output: 5 4 3 2 1

5 4 3 2 1 

---
## 2. MUTABLESEQUENCE

### Overview
A MutableSequence is a Sequence that can be modified after creation.

### Required Methods (in addition to Sequence)

#### `__setitem__(index, value)` - Changes the value at a specific position

In [64]:
#### `__setitem__(index, value)` - Changes the value at a specific position\
class GrowableList:
    def __init__(self, items):
        self._items = list(items)
    
    def __getitem__(self, index):
        return self._items[index]
    
    def __setitem__(self, index, value):
        self._items[index] = value
    
    def __delitem__(self, index):
        del self._items[index]
    
    def __len__(self):
        return len(self._items)
    
    def insert(self, index, value):
        self._items.insert(index, value)

# Usage
gl = GrowableList([1, 2, 3])
gl[1] = 99
print(gl[1])  # Output: 99

99


#### `__delitem__(index)` - Removes an element at a specific position

In [65]:
gl = GrowableList([10, 20, 30, 40])
del gl[2]  # Removes 30
print(len(gl))  # Output: 3

3


#### `insert(index, value)` - Inserts a value at a specific position

In [66]:
gl = GrowableList(['a', 'c', 'd'])
gl.insert(1, 'b')  # Insert 'b' at index 1
print(list(gl))  # Output: ['a', 'b', 'c', 'd']

['a', 'b', 'c', 'd']


### Free Methods
#### `append(value)` - Adds an element to the end

In [68]:
from collections.abc import MutableSequence

class GrowableList(MutableSequence):
    def __init__(self, items):
        self._items = list(items)
    
    def __getitem__(self, index):
        return self._items[index]
    
    def __setitem__(self, index, value):
        self._items[index] = value
    
    def __delitem__(self, index):
        del self._items[index]
    
    def __len__(self):
        return len(self._items)
    
    def insert(self, index, value):
        self._items.insert(index, value)

# Now it works!
gl = GrowableList([1, 2, 3])
gl.append(4)
print(gl[3])  # Output: 4

4


#### `reverse()` - Reverses the sequence in place

In [69]:
gl = GrowableList([1, 2, 3, 4])
gl.reverse()
print(list(gl))  # Output: [4, 3, 2, 1]

[4, 3, 2, 1]


#### `extend(iterable)` - Adds multiple elements to the end

In [70]:
gl = GrowableList([1, 2])
gl.extend([3, 4, 5])
print(list(gl))  # Output: [1, 2, 3, 4, 5]

[1, 2, 3, 4, 5]


#### `pop(index=-1)` - Removes and returns an element (last by default)

In [71]:
gl = GrowableList([10, 20, 30])
value = gl.pop()    # Removes last element
print(value)        # Output: 30
value = gl.pop(0)   # Removes first element
print(value)        # Output: 10

30
10


#### `remove(value)` - Removes the first occurrence of a value

In [72]:
gl = GrowableList(['x', 'y', 'z', 'y'])
gl.remove('y')  # Removes first 'y'
print(list(gl))  # Output: ['x', 'z', 'y']

['x', 'z', 'y']


---
## 3. SET

### Overview
A Set is an unordered collection of unique elements.

### Required Methods

#### `__contains__(value)` - Checks if an element exists in the set

In [73]:
class BasicSet:
    def __init__(self, items):
        self._items = set(items)
    
    def __contains__(self, value):
        return value in self._items
    
    def __iter__(self):
        return iter(self._items)
    
    def __len__(self):
        return len(self._items)

# Usage
bs = BasicSet([1, 2, 3, 3, 4])  # Duplicates removed
print(2 in bs)   # Output: True
print(10 in bs)  # Output: False

True
False


#### `__iter__()` - Enables iteration over set elements

In [74]:
bs = BasicSet(['apple', 'banana', 'cherry'])
for fruit in bs:
    print(fruit)
# Output: apple, banana, cherry (order may vary)

apple
cherry
banana


#### `__len__()` - Returns the number of elements

In [75]:
bs = BasicSet([1, 2, 2, 3, 3, 3])
print(len(bs))  # Output: 3 (only unique elements)

3


### Free Methods (Set Operations)
#### Set comparison operations: `<=`, `<`, `==`, `>=`, `>`

In [76]:
set1 = BasicSet([1, 2])
set2 = BasicSet([1, 2, 3, 4])
# set1 <= set2 would return True (subset)

set3 = BasicSet([1, 2, 3])
set4 = BasicSet([3, 2, 1])
# set3 == set4 would return True (same elements)

#### `&` (intersection) - Returns elements common to both sets

In [77]:
set1 = BasicSet([1, 2, 3, 4])
set2 = BasicSet([3, 4, 5, 6])
# set1 & set2 would return {3, 4}

#### `|` (union) - Returns all elements from both sets

In [78]:
#### `|` (union) - Returns all elements from both sets
set1 = BasicSet([1, 2, 3])
set2 = BasicSet([3, 4, 5])
# set1 | set2 would return {1, 2, 3, 4, 5}

#### `-` (difference) - Returns elements in first set but not in second

In [79]:
set1 = BasicSet([1, 2, 3, 4])
set2 = BasicSet([3, 4, 5])
# set1 - set2 would return {1, 2}

#### `^` (symmetric difference) - Returns elements in either set but not both

In [80]:
set1 = BasicSet([1, 2, 3])
set2 = BasicSet([2, 3, 4])
# set1 ^ set2 would return {1, 4}

---
## 4. MUTABLESET

### Overview
A MutableSet is a Set that can be modified after creation.

### Required Methods (in addition to Set)

#### `add(value)` - Adds an element to the set

In [81]:
class FlexibleSet:
    def __init__(self, items=None):
        self._items = set(items) if items else set()
    
    def __contains__(self, value):
        return value in self._items
    
    def __iter__(self):
        return iter(self._items)
    
    def __len__(self):
        return len(self._items)
    
    def add(self, value):
        self._items.add(value)
    
    def discard(self, value):
        self._items.discard(value)

# Usage
fs = FlexibleSet([1, 2, 3])
fs.add(4)
print(4 in fs)  # Output: True
fs.add(2)       # No effect (already exists)
print(len(fs))  # Output: 4

True
4


#### `discard(value)` - Removes an element if it exists (no error if absent)

In [82]:
fs = FlexibleSet([10, 20, 30])
fs.discard(20)    # Removes 20
fs.discard(999)   # No error, does nothing
print(list(fs))   # Output: [10, 30] (order may vary)

[10, 30]


### Free Methods


#### `clear()` - Removes all elements from the set

fs = FlexibleSet([1, 2, 3, 4, 5])
fs.clear()
print(len(fs))  # Output: 0

#### `pop()` - Removes and returns an arbitrary element

In [84]:
from collections.abc import MutableSet

class FlexibleSet(MutableSet):
    def __init__(self, items=None):
        self._items = set(items) if items else set()
    
    def __contains__(self, value):
        return value in self._items
    
    def __iter__(self):
        return iter(self._items)
    
    def __len__(self):
        return len(self._items)
    
    def add(self, value):
        self._items.add(value)
    
    def discard(self, value):
        self._items.discard(value)

# Now it works!
fs = FlexibleSet([1, 2, 3])
value = fs.pop()
print(value)    # Output: 1, 2, or 3 (arbitrary)
print(len(fs))  # Output: 2

1
2


In [86]:
from collections.abc import Sequence, MutableSequence, Set, MutableSet, Mapping, MutableMapping

class MySequence(Sequence):          # Inherit to get free methods
    def __init__(self, items):
        self._items = list(items)
    
    def __getitem__(self, index):
        return self._items[index]
    
    def __len__(self):
        return len(self._items)

class MyMutableSequence(MutableSequence):  # Inherit to get free methods
    def __init__(self, items):
        self._items = list(items)
    
    def __getitem__(self, index):
        return self._items[index]
    
    def __setitem__(self, index, value):
        self._items[index] = value
    
    def __delitem__(self, index):
        del self._items[index]
    
    def __len__(self):
        return len(self._items)
    
    def insert(self, index, value):
        self._items.insert(index, value)

class MySet(Set):                    # Inherit to get free methods
    def __init__(self, items):
        self._items = set(items)
    
    def __contains__(self, value):
        return value in self._items
    
    def __iter__(self):
        return iter(self._items)
    
    def __len__(self):
        return len(self._items)

class MyMutableSet(MutableSet):      # Inherit to get free methods
    def __init__(self, items=None):
        self._items = set(items) if items else set()
    
    def __contains__(self, value):
        return value in self._items
    
    def __iter__(self):
        return iter(self._items)
    
    def __len__(self):
        return len(self._items)
    
    def add(self, value):
        self._items.add(value)
    
    def discard(self, value):
        self._items.discard(value)

class MyMapping(Mapping):            # Inherit to get free methods
    def __init__(self, pairs=None):
        self._data = dict(pairs) if pairs else {}
    
    def __getitem__(self, key):
        return self._data[key]
    
    def __iter__(self):
        return iter(self._data)
    
    def __len__(self):
        return len(self._data)

class MyMutableMapping(MutableMapping):  # Inherit to get free methods
    def __init__(self, pairs=None):
        self._data = dict(pairs) if pairs else {}
    
    def __getitem__(self, key):
        return self._data[key]
    
    def __setitem__(self, key, value):
        self._data[key] = value
    
    def __delitem__(self, key):
        del self._data[key]
    
    def __iter__(self):
        return iter(self._data)
    
    def __len__(self):
        return len(self._data)


# Test all classes
print("=== Testing MySequence ===")
seq = MySequence([1, 2, 3, 4, 5])
print(f"seq[2] = {seq[2]}")
print(f"len(seq) = {len(seq)}")
print(f"3 in seq = {3 in seq}")
print(f"seq.count(2) = {seq.count(2)}")

print("\n=== Testing MyMutableSequence ===")
mut_seq = MyMutableSequence([1, 2, 3])
mut_seq.append(4)
print(f"After append(4): {list(mut_seq)}")
mut_seq[1] = 99
print(f"After mut_seq[1] = 99: {list(mut_seq)}")
mut_seq.reverse()
print(f"After reverse(): {list(mut_seq)}")

print("\n=== Testing MySet ===")
my_set = MySet([1, 2, 3, 4])
print(f"2 in my_set = {2 in my_set}")
print(f"len(my_set) = {len(my_set)}")
other_set = MySet([3, 4, 5, 6])
# print(f"my_set & other_set = {my_set & other_set}")  # Intersection

print("\n=== Testing MyMutableSet ===")
mut_set = MyMutableSet([1, 2, 3])
mut_set.add(4)
print(f"After add(4): {list(mut_set)}")
mut_set.discard(2)
print(f"After discard(2): {list(mut_set)}")
value = mut_set.pop()
print(f"Popped value: {value}, remaining: {list(mut_set)}")

print("\n=== Testing MyMapping ===")
mapping = MyMapping({'a': 1, 'b': 2, 'c': 3})
print(f"mapping['b'] = {mapping['b']}")
print(f"mapping.get('d', 0) = {mapping.get('d', 0)}")
print(f"mapping.keys() = {list(mapping.keys())}")
print(f"'a' in mapping = {'a' in mapping}")

print("\n=== Testing MyMutableMapping ===")
mut_map = MyMutableMapping({'x': 10, 'y': 20})
mut_map['z'] = 30
print(f"After mut_map['z'] = 30: {dict(mut_map)}")
popped = mut_map.pop('y')
print(f"Popped 'y' = {popped}, remaining: {dict(mut_map)}")
mut_map.update({'a': 100, 'b': 200})
print(f"After update: {dict(mut_map)}")

=== Testing MySequence ===
seq[2] = 3
len(seq) = 5
3 in seq = True
seq.count(2) = 1

=== Testing MyMutableSequence ===
After append(4): [1, 2, 3, 4]
After mut_seq[1] = 99: [1, 99, 3, 4]
After reverse(): [4, 3, 99, 1]

=== Testing MySet ===
2 in my_set = True
len(my_set) = 4

=== Testing MyMutableSet ===
After add(4): [1, 2, 3, 4]
After discard(2): [1, 3, 4]
Popped value: 1, remaining: [3, 4]

=== Testing MyMapping ===
mapping['b'] = 2
mapping.get('d', 0) = 0
mapping.keys() = ['a', 'b', 'c']
'a' in mapping = True

=== Testing MyMutableMapping ===
After mut_map['z'] = 30: {'x': 10, 'y': 20, 'z': 30}
Popped 'y' = 20, remaining: {'x': 10, 'z': 30}
After update: {'x': 10, 'z': 30, 'a': 100, 'b': 200}


#### `remove(value)` - Removes an element (raises error if not found)

In [87]:
fs = FlexibleSet([10, 20, 30])
fs.remove(20)  # Removes 20
# fs.remove(999)  # Would raise KeyError

#### In-place set operations: `|=`, `&=`, `^=`, `-=`

In [88]:
# |= (union update)
fs = FlexibleSet([1, 2, 3])
fs |= FlexibleSet([3, 4, 5])
print(list(fs))  # Output: [1, 2, 3, 4, 5]

# &= (intersection update)
fs = FlexibleSet([1, 2, 3, 4])
fs &= FlexibleSet([2, 3, 5])
print(list(fs))  # Output: [2, 3]

# ^= (symmetric difference update)
fs = FlexibleSet([1, 2, 3])
fs ^= FlexibleSet([2, 3, 4])
print(list(fs))  # Output: [1, 4]

# -= (difference update)
fs = FlexibleSet([1, 2, 3, 4])
fs -= FlexibleSet([2, 4])
print(list(fs))  # Output: [1, 3]

[1, 2, 3, 4, 5]
[2, 3]
[1, 4]
[1, 3]


---
## 5. MAPPING

### Overview
A Mapping is a collection of key-value pairs with unique keys.

### Required Methods

#### `__getitem__(key)` - Retrieves the value associated with a key

In [89]:
class SimpleDict:
    def __init__(self, pairs=None):
        self._data = dict(pairs) if pairs else {}
    
    def __getitem__(self, key):
        return self._data[key]
    
    def __iter__(self):
        return iter(self._data)
    
    def __len__(self):
        return len(self._data)

# Usage
sd = SimpleDict([('name', 'Alice'), ('age', 30)])
print(sd['name'])  # Output: Alice

Alice


#### `__iter__()` - Iterates over the keys

In [90]:
sd = SimpleDict({'a': 1, 'b': 2, 'c': 3})
for key in sd:
    print(key)
# Output: a, b, c

a
b
c


#### `__len__()` - Returns the number of key-value pairs

In [91]:
sd = SimpleDict({'x': 10, 'y': 20, 'z': 30})
print(len(sd))  # Output: 3

3


### Free Methods

#### `get(key, default=None)` - Retrieves a value with a fallback default

In [93]:
from collections.abc import Mapping

class SimpleDict(Mapping):
    def __init__(self, pairs=None):
        self._data = dict(pairs) if pairs else {}
    
    def __getitem__(self, key):
        return self._data[key]
    
    def __iter__(self):
        return iter(self._data)
    
    def __len__(self):
        return len(self._data)

# Now it works!
sd = SimpleDict({'color': 'red', 'size': 'large'})
print(sd.get('color'))        # Output: red
print(sd.get('weight', 0))    # Output: 0 (default)

red
0


#### `keys()` - Returns a view of all keys

In [94]:
sd = SimpleDict({'a': 1, 'b': 2, 'c': 3})
print(list(sd.keys()))  # Output: ['a', 'b', 'c']

['a', 'b', 'c']


#### `items()` - Returns a view of all key-value pairs

In [95]:
sd = SimpleDict({'name': 'Bob', 'score': 95})
for key, value in sd.items():
    print(f"{key}: {value}")
# Output:
# name: Bob
# score: 95

name: Bob
score: 95


#### `values()` - Returns a view of all values

In [96]:
sd = SimpleDict({'x': 100, 'y': 200, 'z': 300})
print(list(sd.values()))  # Output: [100, 200, 300]

[100, 200, 300]


#### `__contains__(key)` - Checks if a key exists (enables `in` operator)

In [97]:
sd = SimpleDict({'apple': 5, 'banana': 3})
print('apple' in sd)   # Output: True
print('orange' in sd)  # Output: False

True
False


---
## 6. MUTABLEMAPPING

### Overview
A MutableMapping is a Mapping that can be modified after creation.

### Required Methods (in addition to Mapping)

#### `__setitem__(key, value)` - Sets or updates a key-value pair

In [98]:
class FlexibleDict:
    def __init__(self, pairs=None):
        self._data = dict(pairs) if pairs else {}
    
    def __getitem__(self, key):
        return self._data[key]
    
    def __setitem__(self, key, value):
        self._data[key] = value
    
    def __delitem__(self, key):
        del self._data[key]
    
    def __iter__(self):
        return iter(self._data)
    
    def __len__(self):
        return len(self._data)

# Usage
fd = FlexibleDict({'x': 1, 'y': 2})
fd['z'] = 3      # Add new pair
fd['x'] = 10     # Update existing
print(fd['x'])   # Output: 10

10


#### `__delitem__(key)` - Removes a key-value pair

In [99]:
fd = FlexibleDict({'a': 1, 'b': 2, 'c': 3})
del fd['b']
print(len(fd))  # Output: 2

2


### Free Methods

#### `pop(key, default=None)` - Removes and returns a value for a key

In [105]:
fd = FlexibleDict({'name': 'Alice', 'age': 30})
age = fd.pop('age')
print(age)         # Output: 30
print(len(fd))     # Output: 1
missing = fd.pop('height', 0)
print(missing)     # Output: 0

30
1
0


#### `popitem()` - Removes and returns an arbitrary key-value pair

In [106]:
fd = FlexibleDict({'x': 100, 'y': 200})
pair = fd.popitem()
print(pair)  # Output: ('x', 100) or ('y', 200)

('x', 100)


#### `clear()` - Removes all key-value pairs

In [101]:
from collections.abc import MutableMapping

class FlexibleDict(MutableMapping):
    def __init__(self, pairs=None):
        self._data = dict(pairs) if pairs else {}
    
    def __getitem__(self, key):
        return self._data[key]
    
    def __setitem__(self, key, value):
        self._data[key] = value
    
    def __delitem__(self, key):
        del self._data[key]
    
    def __iter__(self):
        return iter(self._data)
    
    def __len__(self):
        return len(self._data)

# Now it works!
fd = FlexibleDict({'a': 1, 'b': 2, 'c': 3})
fd.clear()
print(len(fd))  # Output: 0

0


#### `update(other)` - Updates the mapping with key-value pairs from another mapping

In [103]:
fd = FlexibleDict({'a': 1, 'b': 2})
fd.update({'b': 20, 'c': 30})  # Update b, add c
print(fd['b'])  # Output: 20
print(fd['c'])  # Output: 30

20
30


#### `setdefault(key, default=None)` - Returns value if key exists, otherwise sets and returns default

In [104]:
fd = FlexibleDict({'count': 5})
val1 = fd.setdefault('count', 0)
print(val1)  # Output: 5 (key already exists)

val2 = fd.setdefault('total', 0)
print(val2)  # Output: 0 (new key created)
print(fd['total'])  # Output: 0

5
0
0


---
## Summary

| ABC | Required Methods | Key Characteristic |
|-----|-----------------|--------------------|
| **Sequence** | `__getitem__`, `__len__` | Ordered, indexed access |
| **MutableSequence** | + `__setitem__`, `__delitem__`, `insert` | Modifiable ordered collection |
| **Set** | `__contains__`, `__iter__`, `__len__` | Unordered, unique elements |
| **MutableSet** | + `add`, `discard` | Modifiable unique collection |
| **Mapping** | `__getitem__`, `__iter__`, `__len__` | Key-value associations |
| **MutableMapping** | + `__setitem__`, `__delitem__` | Modifiable key-value pairs |

---

## 🎯 Chapter 5 Summary

### Best Practices Covered:

1. **Item 37**: Compose classes instead of nesting built-in types
2. **Item 38**: Use functions for simple interfaces, `__call__` for stateful ones
3. **Item 39**: Use `@classmethod` for generic object construction
4. **Item 40**: Always use `super()` to initialize parent classes
5. **Item 41**: Use mix-ins to compose functionality
6. **Item 42**: Prefer public/protected attributes over private
7. **Item 43**: Inherit from `collections.abc` for custom containers

### Key Principles:

✅ **Composition over Complexity**: Break nested structures into classes  
✅ **Polymorphism**: Leverage class and instance method polymorphism  
✅ **Proper Inheritance**: Use `super()` and understand MRO  
✅ **Flexibility**: Keep code extensible with public APIs  
✅ **Standards**: Use built-in abstract base classes